# データサイエンス100本ノック（構造化データ加工編） - Rust

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [135]:
:dep polars = { version = "0.31.1", features = ["describe", "lazy", "object", "dtype-categorical", "dtype-full", "mode", "temporal", "concat_str", "to_dummies", "log", "dtype-duration", "trigonometry", "polars-io"]}
// :dep smartcore = { version = "0.3.2", default-features = false, features = ["ndarray-bindings", "datasets"]}

In [136]:
use polars::prelude::*;
// データ型の指定
let mut schema = Schema::new();
schema.with_column("customer_id".into(), DataType::Utf8);
schema.with_column("gender_cd".into(), DataType::Utf8);
schema.with_column("postal_cd".into(), DataType::Utf8);
schema.with_column("application_store_cd".into(), DataType::Utf8);
schema.with_column("status_cd".into(), DataType::Utf8);
schema.with_column("category_major_cd".into(), DataType::Utf8);
schema.with_column("category_medium_cd".into(), DataType::Utf8);
schema.with_column("category_small_cd".into(), DataType::Utf8);
schema.with_column("product_cd".into(), DataType::Utf8);
schema.with_column("store_cd".into(), DataType::Utf8);
schema.with_column("prefecture_cd".into(), DataType::Utf8);
schema.with_column("tel_no".into(), DataType::Utf8);
schema.with_column("postal_cd".into(), DataType::Utf8);
schema.with_column("street".into(), DataType::Utf8);
schema.with_column(
    "birth_day".into(),
    DataType::Date
);
schema.with_column(
    "application_date".into(),
    DataType::Utf8
);

let df_customer = LazyCsvReader::new("../data/customer.csv")
    .has_header(true)
    .with_dtype_overwrite(Some(&schema.clone()))
    .finish()
    .unwrap();
let df_category = LazyCsvReader::new("../data/category.csv")
    .has_header(true)
    .with_dtype_overwrite(Some(&schema.clone()))
    .finish()
    .unwrap();
let df_product = LazyCsvReader::new("../data/product.csv")
    .has_header(true)
    .with_dtype_overwrite(Some(&schema.clone()))
    .finish()
    .unwrap();
let df_receipt = LazyCsvReader::new("../data/receipt.csv")
    .has_header(true)
    .with_dtype_overwrite(Some(&schema.clone()))
    .finish()
    .unwrap();
let df_store = LazyCsvReader::new("../data/store.csv")
    .has_header(true)
    .with_dtype_overwrite(Some(&schema.clone()))
    .finish()
    .unwrap();
let df_geocode = LazyCsvReader::new("../data/geocode.csv")
    .has_header(true)
    .with_dtype_overwrite(Some(&schema.clone()))
    .finish()
    .unwrap();

# 演習問題

---
> P-001: レシート明細データ（df_receipt）から全項目の先頭10件を表示し、どのようなデータを保有しているか目視で確認せよ。

In [34]:
df_receipt.clone().collect().unwrap().head(Some(10))

shape: (10, 9)
┌───────────┬────────────┬──────────┬────────────┬───┬────────────┬────────────┬──────────┬────────┐
│ sales_ymd ┆ sales_epoc ┆ store_cd ┆ receipt_no ┆ … ┆ customer_i ┆ product_cd ┆ quantity ┆ amount │
│ ---       ┆ h          ┆ ---      ┆ ---        ┆   ┆ d          ┆ ---        ┆ ---      ┆ ---    │
│ i64       ┆ ---        ┆ str      ┆ i64        ┆   ┆ ---        ┆ str        ┆ i64      ┆ i64    │
│           ┆ i64        ┆          ┆            ┆   ┆ str        ┆            ┆          ┆        │
╞═══════════╪════════════╪══════════╪════════════╪═══╪════════════╪════════════╪══════════╪════════╡
│ 20181103  ┆ 1541203200 ┆ S14006   ┆ 112        ┆ … ┆ CS00621400 ┆ P070305012 ┆ 1        ┆ 158    │
│           ┆            ┆          ┆            ┆   ┆ 0001       ┆            ┆          ┆        │
│ 20181118  ┆ 1542499200 ┆ S13008   ┆ 1132       ┆ … ┆ CS00841500 ┆ P070701017 ┆ 1        ┆ 81     │
│           ┆            ┆          ┆            ┆   ┆ 0097       ┆         

---
> P-002: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。

In [35]:

df_receipt
    .clone()
    .select([
        col("sales_ymd"),
        col("customer_id"),
        col("product_cd"),
        col("amount")
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 4)
┌───────────┬────────────────┬────────────┬────────┐
│ sales_ymd ┆ customer_id    ┆ product_cd ┆ amount │
│ ---       ┆ ---            ┆ ---        ┆ ---    │
│ i64       ┆ str            ┆ str        ┆ i64    │
╞═══════════╪════════════════╪════════════╪════════╡
│ 20181103  ┆ CS006214000001 ┆ P070305012 ┆ 158    │
│ 20181118  ┆ CS008415000097 ┆ P070701017 ┆ 81     │
│ 20170712  ┆ CS028414000014 ┆ P060101005 ┆ 170    │
│ 20190205  ┆ ZZ000000000000 ┆ P050301001 ┆ 25     │
│ …         ┆ …              ┆ …          ┆ …      │
│ 20181205  ┆ CS024514000042 ┆ P080101005 ┆ 30     │
│ 20190922  ┆ CS040415000178 ┆ P070501004 ┆ 128    │
│ 20170504  ┆ ZZ000000000000 ┆ P071302010 ┆ 770    │
│ 20191010  ┆ CS027514000015 ┆ P071101003 ┆ 680    │
└───────────┴────────────────┴────────────┴────────┘

---
> P-003: レシート明細データ（df_receipt）から売上年月日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、10件表示せよ。ただし、sales_ymdをsales_dateに項目名を変更して抽出すること。

In [36]:
df_receipt
    .clone()
    .select([
        col("sales_ymd").alias("sales_date".into()),
        col("customer_id"),
        col("product_cd"),
        col("amount"),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 4)
┌────────────┬────────────────┬────────────┬────────┐
│ sales_date ┆ customer_id    ┆ product_cd ┆ amount │
│ ---        ┆ ---            ┆ ---        ┆ ---    │
│ i64        ┆ str            ┆ str        ┆ i64    │
╞════════════╪════════════════╪════════════╪════════╡
│ 20181103   ┆ CS006214000001 ┆ P070305012 ┆ 158    │
│ 20181118   ┆ CS008415000097 ┆ P070701017 ┆ 81     │
│ 20170712   ┆ CS028414000014 ┆ P060101005 ┆ 170    │
│ 20190205   ┆ ZZ000000000000 ┆ P050301001 ┆ 25     │
│ …          ┆ …              ┆ …          ┆ …      │
│ 20181205   ┆ CS024514000042 ┆ P080101005 ┆ 30     │
│ 20190922   ┆ CS040415000178 ┆ P070501004 ┆ 128    │
│ 20170504   ┆ ZZ000000000000 ┆ P071302010 ┆ 770    │
│ 20191010   ┆ CS027514000015 ┆ P071101003 ┆ 680    │
└────────────┴────────────────┴────────────┴────────┘

---
> P-004: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"

In [37]:
df_receipt
    .clone()
    .select([
        col("sales_ymd"),
        col("customer_id"),
        col("product_cd"),
        col("amount"),
    ])
    .filter(col("customer_id").eq(lit("CS018205000001")))
    .collect()
    .unwrap()

shape: (12, 4)
┌───────────┬────────────────┬────────────┬────────┐
│ sales_ymd ┆ customer_id    ┆ product_cd ┆ amount │
│ ---       ┆ ---            ┆ ---        ┆ ---    │
│ i64       ┆ str            ┆ str        ┆ i64    │
╞═══════════╪════════════════╪════════════╪════════╡
│ 20180911  ┆ CS018205000001 ┆ P071401012 ┆ 2200   │
│ 20180414  ┆ CS018205000001 ┆ P060104007 ┆ 600    │
│ 20170614  ┆ CS018205000001 ┆ P050206001 ┆ 990    │
│ 20170614  ┆ CS018205000001 ┆ P060702015 ┆ 108    │
│ …         ┆ …              ┆ …          ┆ …      │
│ 20190226  ┆ CS018205000001 ┆ P071401020 ┆ 2200   │
│ 20180911  ┆ CS018205000001 ┆ P071401005 ┆ 1100   │
│ 20190216  ┆ CS018205000001 ┆ P040101002 ┆ 218    │
│ 20190924  ┆ CS018205000001 ┆ P091503001 ┆ 280    │
└───────────┴────────────────┴────────────┴────────┘

---
> P-005: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上

In [38]:
df_receipt
    .clone()
    .select([
        col("sales_ymd"),
        col("customer_id"),
        col("product_cd"),
        col("amount"),
    ])
    .filter(
        col("customer_id")
            .eq(lit("CS018205000001"))
            .and(col("amount").gt_eq(lit(1000)))
    )
    .collect()
    .unwrap()

shape: (3, 4)
┌───────────┬────────────────┬────────────┬────────┐
│ sales_ymd ┆ customer_id    ┆ product_cd ┆ amount │
│ ---       ┆ ---            ┆ ---        ┆ ---    │
│ i64       ┆ str            ┆ str        ┆ i64    │
╞═══════════╪════════════════╪════════════╪════════╡
│ 20180911  ┆ CS018205000001 ┆ P071401012 ┆ 2200   │
│ 20190226  ┆ CS018205000001 ┆ P071401020 ┆ 2200   │
│ 20180911  ┆ CS018205000001 ┆ P071401005 ┆ 1100   │
└───────────┴────────────────┴────────────┴────────┘

---
> P-006: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上数量（quantity）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上または売上数量（quantity）が5以上

In [39]:
df_receipt
    .clone()
    .select([
        col("sales_ymd"),
        col("customer_id"),
        col("product_cd"),
        col("quantity"),
        col("amount"),
    ])
    .filter(
        col("customer_id").eq(lit("CS018205000001")).and(
            col("amount")
                .gt_eq(lit(1000))
                .or(col("quantity").gt_eq(lit(5)))
        )
    )
    .collect()
    .unwrap()

shape: (5, 5)
┌───────────┬────────────────┬────────────┬──────────┬────────┐
│ sales_ymd ┆ customer_id    ┆ product_cd ┆ quantity ┆ amount │
│ ---       ┆ ---            ┆ ---        ┆ ---      ┆ ---    │
│ i64       ┆ str            ┆ str        ┆ i64      ┆ i64    │
╞═══════════╪════════════════╪════════════╪══════════╪════════╡
│ 20180911  ┆ CS018205000001 ┆ P071401012 ┆ 1        ┆ 2200   │
│ 20180414  ┆ CS018205000001 ┆ P060104007 ┆ 6        ┆ 600    │
│ 20170614  ┆ CS018205000001 ┆ P050206001 ┆ 5        ┆ 990    │
│ 20190226  ┆ CS018205000001 ┆ P071401020 ┆ 1        ┆ 2200   │
│ 20180911  ┆ CS018205000001 ┆ P071401005 ┆ 1        ┆ 1100   │
└───────────┴────────────────┴────────────┴──────────┴────────┘

---
> P-007: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 売上金額（amount）が1,000以上2,000以下

In [40]:
df_receipt
    .clone()
    .select([
        col("sales_ymd"),
        col("customer_id"),
        col("product_cd"),
        col("amount"),
    ])
    .filter(
        col("customer_id").eq(lit("CS018205000001")).and(
            col("amount")
                .gt_eq(lit(1000))
                .and(col("amount").lt_eq(lit(2000)))
        )
    )
    .collect()
    .unwrap()

shape: (1, 4)
┌───────────┬────────────────┬────────────┬────────┐
│ sales_ymd ┆ customer_id    ┆ product_cd ┆ amount │
│ ---       ┆ ---            ┆ ---        ┆ ---    │
│ i64       ┆ str            ┆ str        ┆ i64    │
╞═══════════╪════════════════╪════════════╪════════╡
│ 20180911  ┆ CS018205000001 ┆ P071401005 ┆ 1100   │
└───────────┴────────────────┴────────────┴────────┘

---
> P-008: レシート明細データ（df_receipt）から売上日（sales_ymd）、顧客ID（customer_id）、商品コード（product_cd）、売上金額（amount）の順に列を指定し、以下の全ての条件を満たすデータを抽出せよ。
> - 顧客ID（customer_id）が"CS018205000001"
> - 商品コード（product_cd）が"P071401019"以外

In [41]:
df_receipt
    .clone()
    .select([
        col("sales_ymd"),
        col("customer_id"),
        col("product_cd"),
        col("amount"),
    ])
    .filter(
        col("customer_id")
            .eq(lit("CS018205000001"))
            .and(col("product_cd").ne(&lit("P071401019")))
    )
    .collect()
    .unwrap()

shape: (12, 4)
┌───────────┬────────────────┬────────────┬────────┐
│ sales_ymd ┆ customer_id    ┆ product_cd ┆ amount │
│ ---       ┆ ---            ┆ ---        ┆ ---    │
│ i64       ┆ str            ┆ str        ┆ i64    │
╞═══════════╪════════════════╪════════════╪════════╡
│ 20180911  ┆ CS018205000001 ┆ P071401012 ┆ 2200   │
│ 20180414  ┆ CS018205000001 ┆ P060104007 ┆ 600    │
│ 20170614  ┆ CS018205000001 ┆ P050206001 ┆ 990    │
│ 20170614  ┆ CS018205000001 ┆ P060702015 ┆ 108    │
│ …         ┆ …              ┆ …          ┆ …      │
│ 20190226  ┆ CS018205000001 ┆ P071401020 ┆ 2200   │
│ 20180911  ┆ CS018205000001 ┆ P071401005 ┆ 1100   │
│ 20190216  ┆ CS018205000001 ┆ P040101002 ┆ 218    │
│ 20190924  ┆ CS018205000001 ┆ P091503001 ┆ 280    │
└───────────┴────────────────┴────────────┴────────┘

---
> P-009: 以下の処理において、出力結果を変えずにORをANDに書き換えよ。
> 
> `df_store.query("not(prefecture_cd == "13" | floor_area > 900)")`

In [42]:
df_store
    .clone()
    .filter(
        col("prefecture_cd")
            .eq(lit("13"))
            .not()
            .and(col("floor_area").gt(lit(900)).not())
    )
    .collect()
    .unwrap()

shape: (3, 10)
┌──────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ store_cd ┆ store_name ┆ prefectur ┆ prefectur ┆ … ┆ tel_no    ┆ longitude ┆ latitude ┆ floor_are │
│ ---      ┆ ---        ┆ e_cd      ┆ e         ┆   ┆ ---       ┆ ---       ┆ ---      ┆ a         │
│ str      ┆ str        ┆ ---       ┆ ---       ┆   ┆ str       ┆ f64       ┆ f64      ┆ ---       │
│          ┆            ┆ str       ┆ str       ┆   ┆           ┆           ┆          ┆ f64       │
╞══════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ S14046   ┆ 北山田店   ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.5916  ┆ 35.56189 ┆ 831.0     │
│          ┆            ┆           ┆           ┆   ┆ 049       ┆           ┆          ┆           │
│ S14011   ┆ 日吉本町店 ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.6316  ┆ 35.54655 ┆ 890.0     │
│          ┆            ┆           ┆           ┆   ┆ 033       ┆           ┆          ┆     

---
> P-010: 店舗データ（df_store）から、店舗コード（store_cd）が"S14"で始まるものだけ全項目抽出し、10件表示せよ。

In [43]:
df_store
    .clone()
    .filter(col("store_cd").str().starts_with(lit("S14")))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 10)
┌──────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ store_cd ┆ store_name ┆ prefectur ┆ prefectur ┆ … ┆ tel_no    ┆ longitude ┆ latitude ┆ floor_are │
│ ---      ┆ ---        ┆ e_cd      ┆ e         ┆   ┆ ---       ┆ ---       ┆ ---      ┆ a         │
│ str      ┆ str        ┆ ---       ┆ ---       ┆   ┆ str       ┆ f64       ┆ f64      ┆ ---       │
│          ┆            ┆ str       ┆ str       ┆   ┆           ┆           ┆          ┆ f64       │
╞══════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ S14010   ┆ 菊名店     ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.6326  ┆ 35.50049 ┆ 1732.0    │
│          ┆            ┆           ┆           ┆   ┆ 032       ┆           ┆          ┆           │
│ S14033   ┆ 阿久和店   ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.4961  ┆ 35.45918 ┆ 1495.0    │
│          ┆            ┆           ┆           ┆   ┆ 043       ┆           ┆          ┆  

---
> P-011: 顧客データ（df_customer）から顧客ID（customer_id）の末尾が1のものだけ全項目抽出し、10件表示せよ。

In [44]:
df_customer
    .clone()
    .filter(col("customer_id").str().ends_with(lit("1")))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 11)
┌────────────┬────────────┬───────────┬────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ customer_n ┆ gender_cd ┆ gender ┆ … ┆ address   ┆ applicati ┆ applicati ┆ status_cd │
│ d          ┆ ame        ┆ ---       ┆ ---    ┆   ┆ ---       ┆ on_store_ ┆ on_date   ┆ ---       │
│ ---        ┆ ---        ┆ str       ┆ str    ┆   ┆ str       ┆ cd        ┆ ---       ┆ str       │
│ str        ┆ str        ┆           ┆        ┆   ┆           ┆ ---       ┆ i64       ┆           │
│            ┆            ┆           ┆        ┆   ┆           ┆ str       ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ CS03761300 ┆ 六角 雅彦  ┆ 9         ┆ 不明   ┆ … ┆ 東京都江  ┆ S13037    ┆ 20150414  ┆ 0-0000000 │
│ 0071       ┆            ┆           ┆        ┆   ┆ 東区南砂* ┆           ┆           ┆ 0-0       │
│            ┆            ┆           ┆        ┆   ┆ ********* ┆           ┆           ┆ 

---
> P-012: 店舗データ（df_store）から、住所 (address) に"横浜市"が含まれるものだけ全項目表示せよ。

In [45]:
df_store
    .clone()
    .filter(col("address").str().contains(lit("横浜市"), true))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 10)
┌──────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ store_cd ┆ store_name ┆ prefectur ┆ prefectur ┆ … ┆ tel_no    ┆ longitude ┆ latitude ┆ floor_are │
│ ---      ┆ ---        ┆ e_cd      ┆ e         ┆   ┆ ---       ┆ ---       ┆ ---      ┆ a         │
│ str      ┆ str        ┆ ---       ┆ ---       ┆   ┆ str       ┆ f64       ┆ f64      ┆ ---       │
│          ┆            ┆ str       ┆ str       ┆   ┆           ┆           ┆          ┆ f64       │
╞══════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ S14010   ┆ 菊名店     ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.6326  ┆ 35.50049 ┆ 1732.0    │
│          ┆            ┆           ┆           ┆   ┆ 032       ┆           ┆          ┆           │
│ S14033   ┆ 阿久和店   ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.4961  ┆ 35.45918 ┆ 1495.0    │
│          ┆            ┆           ┆           ┆   ┆ 043       ┆           ┆          ┆  

---
> P-013: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まるデータを全項目抽出し、10件表示せよ。

In [46]:
df_customer
    .clone()
    .filter(col("status_cd").str().contains(lit("^[A-F]"), true))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 11)
┌────────────┬────────────┬───────────┬────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ customer_n ┆ gender_cd ┆ gender ┆ … ┆ address   ┆ applicati ┆ applicati ┆ status_cd │
│ d          ┆ ame        ┆ ---       ┆ ---    ┆   ┆ ---       ┆ on_store_ ┆ on_date   ┆ ---       │
│ ---        ┆ ---        ┆ str       ┆ str    ┆   ┆ str       ┆ cd        ┆ ---       ┆ str       │
│ str        ┆ str        ┆           ┆        ┆   ┆           ┆ ---       ┆ i64       ┆           │
│            ┆            ┆           ┆        ┆   ┆           ┆ str       ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ CS03141500 ┆ 宇多田     ┆ 1         ┆ 女性   ┆ … ┆ 東京都渋  ┆ S13031    ┆ 20150529  ┆ D-2010032 │
│ 0172       ┆ 貴美子     ┆           ┆        ┆   ┆ 谷区代々  ┆           ┆           ┆ 5-C       │
│            ┆            ┆           ┆        ┆   ┆ 木******* ┆           ┆           ┆    

---
> P-014: 顧客データ（df_customer）から、ステータスコード（status_cd）の末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [47]:
df_customer
    .clone()
    .filter(col("status_cd").str().contains(lit("[1-9]$"), true))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 11)
┌────────────┬────────────┬───────────┬────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ customer_n ┆ gender_cd ┆ gender ┆ … ┆ address   ┆ applicati ┆ applicati ┆ status_cd │
│ d          ┆ ame        ┆ ---       ┆ ---    ┆   ┆ ---       ┆ on_store_ ┆ on_date   ┆ ---       │
│ ---        ┆ ---        ┆ str       ┆ str    ┆   ┆ str       ┆ cd        ┆ ---       ┆ str       │
│ str        ┆ str        ┆           ┆        ┆   ┆           ┆ ---       ┆ i64       ┆           │
│            ┆            ┆           ┆        ┆   ┆           ┆ str       ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ CS00121500 ┆ 田崎 美紀  ┆ 1         ┆ 女性   ┆ … ┆ 東京都大  ┆ S13001    ┆ 20170605  ┆ 6-2009092 │
│ 0145       ┆            ┆           ┆        ┆   ┆ 田区仲六  ┆           ┆           ┆ 9-2       │
│            ┆            ┆           ┆        ┆   ┆ 郷******* ┆           ┆           ┆  

---
> P-015: 顧客データ（df_customer）から、ステータスコード（status_cd）の先頭がアルファベットのA〜Fで始まり、末尾が数字の1〜9で終わるデータを全項目抽出し、10件表示せよ。

In [48]:
df_customer
    .clone()
    .filter(col("status_cd").str().contains(lit("^[A-F].*[0-9]$"), true))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 11)
┌────────────┬────────────┬───────────┬────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ customer_n ┆ gender_cd ┆ gender ┆ … ┆ address   ┆ applicati ┆ applicati ┆ status_cd │
│ d          ┆ ame        ┆ ---       ┆ ---    ┆   ┆ ---       ┆ on_store_ ┆ on_date   ┆ ---       │
│ ---        ┆ ---        ┆ str       ┆ str    ┆   ┆ str       ┆ cd        ┆ ---       ┆ str       │
│ str        ┆ str        ┆           ┆        ┆   ┆           ┆ ---       ┆ i64       ┆           │
│            ┆            ┆           ┆        ┆   ┆           ┆ str       ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ CS01121500 ┆ 芦田 沙耶  ┆ 1         ┆ 女性   ┆ … ┆ 神奈川県  ┆ S14011    ┆ 20150228  ┆ C-2010042 │
│ 0048       ┆            ┆           ┆        ┆   ┆ 横浜市港  ┆           ┆           ┆ 1-9       │
│            ┆            ┆           ┆        ┆   ┆ 北区日吉  ┆           ┆           ┆     

---
> P-016: 店舗データ（df_store）から、電話番号（tel_no）が3桁-3桁-4桁のデータを全項目表示せよ。

In [49]:
df_store
    .clone()
    .filter(
        col("tel_no")
            .str()
            .contains(lit(r"\d{3}-\d{3}-\d{4}"), true)
    )
    .collect()
    .unwrap()

shape: (34, 10)
┌──────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ store_cd ┆ store_name ┆ prefectur ┆ prefectur ┆ … ┆ tel_no    ┆ longitude ┆ latitude ┆ floor_are │
│ ---      ┆ ---        ┆ e_cd      ┆ e         ┆   ┆ ---       ┆ ---       ┆ ---      ┆ a         │
│ str      ┆ str        ┆ ---       ┆ ---       ┆   ┆ str       ┆ f64       ┆ f64      ┆ ---       │
│          ┆            ┆ str       ┆ str       ┆   ┆           ┆           ┆          ┆ f64       │
╞══════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ S12014   ┆ 千草台店   ┆ 12        ┆ 千葉県    ┆ … ┆ 043-123-4 ┆ 140.118   ┆ 35.63559 ┆ 1698.0    │
│          ┆            ┆           ┆           ┆   ┆ 003       ┆           ┆          ┆           │
│ S13002   ┆ 国分寺店   ┆ 13        ┆ 東京都    ┆ … ┆ 042-123-4 ┆ 139.4802  ┆ 35.70566 ┆ 1735.0    │
│          ┆            ┆           ┆           ┆   ┆ 008       ┆           ┆          ┆ 

---
> P-017: 顧客データ（df_customer）を生年月日（birth_day）で高齢順にソートし、先頭から全項目を10件表示せよ。

In [50]:
df_customer
    .clone()
    .sort(
        "birth_day",
        SortOptions {
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true,
        }
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 11)
┌────────────┬────────────┬───────────┬────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ customer_n ┆ gender_cd ┆ gender ┆ … ┆ address   ┆ applicati ┆ applicati ┆ status_cd │
│ d          ┆ ame        ┆ ---       ┆ ---    ┆   ┆ ---       ┆ on_store_ ┆ on_date   ┆ ---       │
│ ---        ┆ ---        ┆ str       ┆ str    ┆   ┆ str       ┆ cd        ┆ ---       ┆ str       │
│ str        ┆ str        ┆           ┆        ┆   ┆           ┆ ---       ┆ i64       ┆           │
│            ┆            ┆           ┆        ┆   ┆           ┆ str       ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ CS00381300 ┆ 村山       ┆ 1         ┆ 女性   ┆ … ┆ 東京都調  ┆ S13003    ┆ 20160214  ┆ 0-0000000 │
│ 0014       ┆ 菜々美     ┆           ┆        ┆   ┆ 布市菊野  ┆           ┆           ┆ 0-0       │
│            ┆            ┆           ┆        ┆   ┆ 台******* ┆           ┆           ┆   

---
> P-018: 顧客データ（df_customer）を生年月日（birth_day）で若い順にソートし、先頭から全項目を10件表示せよ。

In [51]:
df_store
    .clone()
    .filter(col("store_cd").str().starts_with(lit("S14")))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 10)
┌──────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ store_cd ┆ store_name ┆ prefectur ┆ prefectur ┆ … ┆ tel_no    ┆ longitude ┆ latitude ┆ floor_are │
│ ---      ┆ ---        ┆ e_cd      ┆ e         ┆   ┆ ---       ┆ ---       ┆ ---      ┆ a         │
│ str      ┆ str        ┆ ---       ┆ ---       ┆   ┆ str       ┆ f64       ┆ f64      ┆ ---       │
│          ┆            ┆ str       ┆ str       ┆   ┆           ┆           ┆          ┆ f64       │
╞══════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ S14010   ┆ 菊名店     ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.6326  ┆ 35.50049 ┆ 1732.0    │
│          ┆            ┆           ┆           ┆   ┆ 032       ┆           ┆          ┆           │
│ S14033   ┆ 阿久和店   ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.4961  ┆ 35.45918 ┆ 1495.0    │
│          ┆            ┆           ┆           ┆   ┆ 043       ┆           ┆          ┆  

---
> P-019: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合は同一順位を付与するものとする。

In [52]:
df_store
    .clone()
    .filter(col("store_cd").str().starts_with(lit("S14")))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 10)
┌──────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ store_cd ┆ store_name ┆ prefectur ┆ prefectur ┆ … ┆ tel_no    ┆ longitude ┆ latitude ┆ floor_are │
│ ---      ┆ ---        ┆ e_cd      ┆ e         ┆   ┆ ---       ┆ ---       ┆ ---      ┆ a         │
│ str      ┆ str        ┆ ---       ┆ ---       ┆   ┆ str       ┆ f64       ┆ f64      ┆ ---       │
│          ┆            ┆ str       ┆ str       ┆   ┆           ┆           ┆          ┆ f64       │
╞══════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ S14010   ┆ 菊名店     ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.6326  ┆ 35.50049 ┆ 1732.0    │
│          ┆            ┆           ┆           ┆   ┆ 032       ┆           ┆          ┆           │
│ S14033   ┆ 阿久和店   ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.4961  ┆ 35.45918 ┆ 1495.0    │
│          ┆            ┆           ┆           ┆   ┆ 043       ┆           ┆          ┆  

---
> P-020: レシート明細データ（df_receipt）に対し、1件あたりの売上金額（amount）が高い順にランクを付与し、先頭から10件表示せよ。項目は顧客ID（customer_id）、売上金額（amount）、付与したランクを表示させること。なお、売上金額（amount）が等しい場合でも別順位を付与すること。

In [53]:
df_store
    .clone()
    .filter(col("store_cd").str().starts_with(lit("S14")))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 10)
┌──────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ store_cd ┆ store_name ┆ prefectur ┆ prefectur ┆ … ┆ tel_no    ┆ longitude ┆ latitude ┆ floor_are │
│ ---      ┆ ---        ┆ e_cd      ┆ e         ┆   ┆ ---       ┆ ---       ┆ ---      ┆ a         │
│ str      ┆ str        ┆ ---       ┆ ---       ┆   ┆ str       ┆ f64       ┆ f64      ┆ ---       │
│          ┆            ┆ str       ┆ str       ┆   ┆           ┆           ┆          ┆ f64       │
╞══════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ S14010   ┆ 菊名店     ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.6326  ┆ 35.50049 ┆ 1732.0    │
│          ┆            ┆           ┆           ┆   ┆ 032       ┆           ┆          ┆           │
│ S14033   ┆ 阿久和店   ┆ 14        ┆ 神奈川県  ┆ … ┆ 045-123-4 ┆ 139.4961  ┆ 35.45918 ┆ 1495.0    │
│          ┆            ┆           ┆           ┆   ┆ 043       ┆           ┆          ┆  

---
> P-021: レシート明細データ（df_receipt）に対し、件数をカウントせよ。

In [54]:
df_receipt
    .clone()
    .collect()
    .unwrap()
    .height()

104681

---
> P-022: レシート明細データ（df_receipt）の顧客ID（customer_id）に対し、ユニーク件数をカウントせよ。

In [55]:
df_receipt
    .clone()
    .select([col("customer_id").n_unique()])
    .collect()
    .unwrap()
    

shape: (1, 1)
┌─────────────┐
│ customer_id │
│ ---         │
│ u32         │
╞═════════════╡
│ 8307        │
└─────────────┘

---
> P-023: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）と売上数量（quantity）を合計せよ。

In [56]:
df_receipt
    .clone()
    .groupby(["store_cd"])
    .agg([col("amount").sum(), col("quantity").sum()])
    .sort("store_cd", SortOptions{descending: (false), nulls_last: (true), multithreaded: (true), maintain_order: (true)})
    .collect()
    .unwrap()

shape: (52, 3)
┌──────────┬────────┬──────────┐
│ store_cd ┆ amount ┆ quantity │
│ ---      ┆ ---    ┆ ---      │
│ str      ┆ i64    ┆ i64      │
╞══════════╪════════╪══════════╡
│ S12007   ┆ 638761 ┆ 2099     │
│ S12013   ┆ 787513 ┆ 2425     │
│ S12014   ┆ 725167 ┆ 2358     │
│ S12029   ┆ 794741 ┆ 2555     │
│ …        ┆ …      ┆ …        │
│ S14047   ┆ 338329 ┆ 1041     │
│ S14048   ┆ 234276 ┆ 769      │
│ S14049   ┆ 230808 ┆ 788      │
│ S14050   ┆ 167090 ┆ 580      │
└──────────┴────────┴──────────┘

---
> P-024: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）を求め、10件表示せよ。

In [57]:
df_receipt
    .clone()
    .groupby(["customer_id"])
    .agg([col("sales_ymd").max()])
    .sort("customer_id", SortOptions{descending: (false), nulls_last: (true), multithreaded: (true),maintain_order: true,})
    .collect()
    .unwrap()
    .head(Some(10))


shape: (10, 2)
┌────────────────┬───────────┐
│ customer_id    ┆ sales_ymd │
│ ---            ┆ ---       │
│ str            ┆ i64       │
╞════════════════╪═══════════╡
│ CS001113000004 ┆ 20190308  │
│ CS001114000005 ┆ 20190731  │
│ CS001115000010 ┆ 20190405  │
│ CS001205000004 ┆ 20190625  │
│ …              ┆ …         │
│ CS001212000027 ┆ 20170127  │
│ CS001212000031 ┆ 20180906  │
│ CS001212000046 ┆ 20170811  │
│ CS001212000070 ┆ 20191018  │
└────────────────┴───────────┘

---
> P-025: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も古い売上年月日（sales_ymd）を求め、10件表示せよ。

In [58]:
df_receipt
    .clone()
    .groupby(["customer_id"])
    .agg([col("sales_ymd").min()])
    .sort("customer_id", SortOptions{descending: (false), nulls_last: (true), multithreaded: (true), maintain_order: true,})
    .collect()
    .unwrap()
    .head(Some(10))


shape: (10, 2)
┌────────────────┬───────────┐
│ customer_id    ┆ sales_ymd │
│ ---            ┆ ---       │
│ str            ┆ i64       │
╞════════════════╪═══════════╡
│ CS001113000004 ┆ 20190308  │
│ CS001114000005 ┆ 20180503  │
│ CS001115000010 ┆ 20171228  │
│ CS001205000004 ┆ 20170914  │
│ …              ┆ …         │
│ CS001212000027 ┆ 20170127  │
│ CS001212000031 ┆ 20180906  │
│ CS001212000046 ┆ 20170811  │
│ CS001212000070 ┆ 20191018  │
└────────────────┴───────────┘

---
> P-026: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに最も新しい売上年月日（sales_ymd）と古い売上年月日を求め、両者が異なるデータを10件表示せよ。

In [59]:
df_receipt
    .clone()
    .groupby(["customer_id"])
    .agg([col("sales_ymd").max().alias("sales_ymd_max"), col("sales_ymd").min().alias("sales_ymd_min")])
    .filter(col("sales_ymd_max").neq("sales_ymd_min"))
    .sort("customer_id", SortOptions{descending: (false), nulls_last: (true), multithreaded: (true), maintain_order: true,})
    .collect()
    .unwrap()
    .head(Some(10))


shape: (10, 3)
┌────────────────┬───────────────┬───────────────┐
│ customer_id    ┆ sales_ymd_max ┆ sales_ymd_min │
│ ---            ┆ ---           ┆ ---           │
│ str            ┆ i64           ┆ i64           │
╞════════════════╪═══════════════╪═══════════════╡
│ CS001114000005 ┆ 20190731      ┆ 20180503      │
│ CS001115000010 ┆ 20190405      ┆ 20171228      │
│ CS001205000004 ┆ 20190625      ┆ 20170914      │
│ CS001205000006 ┆ 20190224      ┆ 20180207      │
│ …              ┆ …             ┆ …             │
│ CS001214000048 ┆ 20190929      ┆ 20171109      │
│ CS001214000052 ┆ 20190617      ┆ 20180208      │
│ CS001215000005 ┆ 20181021      ┆ 20170206      │
│ CS001215000040 ┆ 20171022      ┆ 20170214      │
└────────────────┴───────────────┴───────────────┘

---
> P-027: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、降順でTOP5を表示せよ。

In [60]:
df_receipt
    .clone()
    .groupby(["store_cd"])
    .agg([col("amount").mean()])
    .sort("amount",
        SortOptions{
            descending: (true),
            nulls_last: (true),
            multithreaded: (true),
            maintain_order: true,
        }
    )
    .collect()
    .unwrap()
    .head(Some(5))

shape: (5, 2)
┌──────────┬────────────┐
│ store_cd ┆ amount     │
│ ---      ┆ ---        │
│ str      ┆ f64        │
╞══════════╪════════════╡
│ S13052   ┆ 402.86747  │
│ S13015   ┆ 351.11196  │
│ S13003   ┆ 350.915519 │
│ S14010   ┆ 348.791262 │
│ S13001   ┆ 348.470386 │
└──────────┴────────────┘

---
> P-028: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の中央値を計算し、降順でTOP5を表示せよ。

In [61]:
df_receipt
    .clone()
    .groupby(["store_cd"])
    .agg([col("amount").median()])
    .sort("amount", SortOptions{descending: (true), nulls_last: (true), multithreaded: (true),maintain_order: true,})
    .collect()
    .unwrap()
    .head(Some(5))

shape: (5, 2)
┌──────────┬────────┐
│ store_cd ┆ amount │
│ ---      ┆ ---    │
│ str      ┆ f64    │
╞══════════╪════════╡
│ S13052   ┆ 190.0  │
│ S14010   ┆ 188.0  │
│ S14050   ┆ 185.0  │
│ S13003   ┆ 180.0  │
│ S13018   ┆ 180.0  │
└──────────┴────────┘

---
> P-029: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに商品コード（product_cd）の最頻値を求め、10件表示させよ。

In [62]:
df_receipt
    .clone()
    .groupby(["store_cd"])
    .agg([col("product_cd").mode().first()])
    .sort("store_cd", SortOptions{descending: (false), nulls_last: (true), multithreaded: (true), maintain_order: true,})
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 2)
┌──────────┬────────────┐
│ store_cd ┆ product_cd │
│ ---      ┆ ---        │
│ str      ┆ str        │
╞══════════╪════════════╡
│ S12007   ┆ P060303001 │
│ S12013   ┆ P060303001 │
│ S12014   ┆ P060303001 │
│ S12029   ┆ P060303001 │
│ …        ┆ …          │
│ S13002   ┆ P060303001 │
│ S13003   ┆ P071401001 │
│ S13004   ┆ P060303001 │
│ S13005   ┆ P040503001 │
└──────────┴────────────┘

---
> P-030: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の分散を計算し、降順で5件表示せよ。

In [63]:
df_receipt
    .clone()
    .groupby(["store_cd"])
    .agg([col("amount").var(0)])
    .sort("amount", SortOptions{descending: (true), nulls_last: (true), multithreaded: (true), maintain_order: true})
    .collect()
    .unwrap()
    .head(Some(5))

shape: (5, 2)
┌──────────┬───────────────┐
│ store_cd ┆ amount        │
│ ---      ┆ ---           │
│ str      ┆ f64           │
╞══════════╪═══════════════╡
│ S13052   ┆ 440088.701311 │
│ S14011   ┆ 306314.558164 │
│ S14034   ┆ 296920.081011 │
│ S13001   ┆ 295431.993329 │
│ S13015   ┆ 295294.361116 │
└──────────┴───────────────┘

---
> P-031: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標準偏差を計算し、降順で5件表示せよ。

TIPS:

PandasとNumpyでddofのデフォルト値が異なることに注意しましょう
```
Pandas：
DataFrame.std(self, axis=None, skipna=None, level=None, ddof=1, numeric_only=None, **kwargs)
Numpy:
numpy.std(a, axis=None, dtype=None, out=None, ddof=0, keepdims=)
```

In [64]:
df_receipt
    .clone()
    .groupby(["store_cd"])
    .agg([col("amount").std(0)])
    .sort("amount", SortOptions{descending: (true), nulls_last: (true), multithreaded: (true), maintain_order: true,})
    .collect()
    .unwrap()
    .head(Some(5))


shape: (5, 2)
┌──────────┬────────────┐
│ store_cd ┆ amount     │
│ ---      ┆ ---        │
│ str      ┆ f64        │
╞══════════╪════════════╡
│ S13052   ┆ 663.391816 │
│ S14011   ┆ 553.456916 │
│ S14034   ┆ 544.903736 │
│ S13001   ┆ 543.536561 │
│ S13015   ┆ 543.409938 │
└──────────┴────────────┘

---
> P-032: レシート明細データ（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [65]:
df_receipt
    .clone()
    .select([
        col("amount").quantile(lit(0.0), QuantileInterpolOptions::Nearest).alias("0%tile"),
        col("amount").quantile(lit(0.25), QuantileInterpolOptions::Nearest).alias("25%tile"),
        col("amount").quantile(lit(0.50), QuantileInterpolOptions::Nearest).alias("50%tile"),
        col("amount").quantile(lit(0.75), QuantileInterpolOptions::Nearest).alias("75%tile"),
        col("amount").quantile(lit(1.00), QuantileInterpolOptions::Nearest).alias("100%tile"),
    ])
    .collect()
    .unwrap()

shape: (1, 5)
┌────────┬─────────┬─────────┬─────────┬──────────┐
│ 0%tile ┆ 25%tile ┆ 50%tile ┆ 75%tile ┆ 100%tile │
│ ---    ┆ ---     ┆ ---     ┆ ---     ┆ ---      │
│ f64    ┆ f64     ┆ f64     ┆ f64     ┆ f64      │
╞════════╪═════════╪═════════╪═════════╪══════════╡
│ 10.0   ┆ 102.0   ┆ 170.0   ┆ 288.0   ┆ 10925.0  │
└────────┴─────────┴─────────┴─────────┴──────────┘

---
> P-033: レシート明細データ（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [66]:
df_receipt
    .clone()
    .groupby(["store_cd"])
    .agg([col("amount").mean()])
    .filter(col("amount").gt_eq(lit(330)))
    .sort("store_cd", SortOptions{descending: (false), nulls_last: (true), multithreaded: (true), maintain_order: true,})
    .collect()
    .unwrap()

shape: (13, 2)
┌──────────┬────────────┐
│ store_cd ┆ amount     │
│ ---      ┆ ---        │
│ str      ┆ f64        │
╞══════════╪════════════╡
│ S12013   ┆ 330.19413  │
│ S13001   ┆ 348.470386 │
│ S13003   ┆ 350.915519 │
│ S13004   ┆ 330.943949 │
│ …        ┆ …          │
│ S14011   ┆ 335.718333 │
│ S14026   ┆ 332.340588 │
│ S14045   ┆ 330.082073 │
│ S14047   ┆ 330.077073 │
└──────────┴────────────┘

---
> P-034: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [67]:
df_receipt
    .clone()
    .filter(col("customer_id").str().starts_with(lit("Z")).not())
    .groupby(["customer_id"]).agg([col("amount").sum()])
    .select([col("amount").mean()])
    .collect()
    .unwrap()

shape: (1, 1)
┌─────────────┐
│ amount      │
│ ---         │
│ f64         │
╞═════════════╡
│ 2547.742235 │
└─────────────┘

---
> P-035: レシート明細データ（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出し、10件表示せよ。ただし、顧客IDが"Z"から始まるものは非会員を表すため、除外して計算すること。

In [68]:
df_receipt
    .clone()
    .filter(col("customer_id").str().starts_with(lit("Z")).not())
    .groupby(["customer_id"]).agg([col("amount").sum()])
    .with_column(col("amount").mean().alias("avg_amount"))
    .filter(col("amount").gt_eq(col("avg_amount")))
    .sort("customer_id", SortOptions{descending: (false), nulls_last: (true), multithreaded: (true), maintain_order: true,})
    .collect()
    .unwrap()
    .head(Some(10))


shape: (10, 3)
┌────────────────┬────────┬─────────────┐
│ customer_id    ┆ amount ┆ avg_amount  │
│ ---            ┆ ---    ┆ ---         │
│ str            ┆ i64    ┆ f64         │
╞════════════════╪════════╪═════════════╡
│ CS001115000010 ┆ 3044   ┆ 2547.742235 │
│ CS001205000006 ┆ 3337   ┆ 2547.742235 │
│ CS001214000009 ┆ 4685   ┆ 2547.742235 │
│ CS001214000017 ┆ 4132   ┆ 2547.742235 │
│ …              ┆ …      ┆ …           │
│ CS001304000006 ┆ 3726   ┆ 2547.742235 │
│ CS001305000005 ┆ 3485   ┆ 2547.742235 │
│ CS001305000011 ┆ 4370   ┆ 2547.742235 │
│ CS001315000180 ┆ 3300   ┆ 2547.742235 │
└────────────────┴────────┴─────────────┘

---
> P-036: レシート明細データ（df_receipt）と店舗データ（df_store）を内部結合し、レシート明細データの全項目と店舗データの店舗名（store_name）を10件表示せよ。

In [69]:
df_receipt
    .clone()
    .inner_join(
        df_store.clone().select([col("store_cd"), col("store_name")]),
        col("store_cd"),
        col("store_cd"),
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 10)
┌───────────┬────────────┬──────────┬────────────┬───┬────────────┬──────────┬────────┬────────────┐
│ sales_ymd ┆ sales_epoc ┆ store_cd ┆ receipt_no ┆ … ┆ product_cd ┆ quantity ┆ amount ┆ store_name │
│ ---       ┆ h          ┆ ---      ┆ ---        ┆   ┆ ---        ┆ ---      ┆ ---    ┆ ---        │
│ i64       ┆ ---        ┆ str      ┆ i64        ┆   ┆ str        ┆ i64      ┆ i64    ┆ str        │
│           ┆ i64        ┆          ┆            ┆   ┆            ┆          ┆        ┆            │
╞═══════════╪════════════╪══════════╪════════════╪═══╪════════════╪══════════╪════════╪════════════╡
│ 20181103  ┆ 1541203200 ┆ S14006   ┆ 112        ┆ … ┆ P070305012 ┆ 1        ┆ 158    ┆ 葛が谷店   │
│ 20181118  ┆ 1542499200 ┆ S13008   ┆ 1132       ┆ … ┆ P070701017 ┆ 1        ┆ 81     ┆ 成城店     │
│ 20170712  ┆ 1499817600 ┆ S14028   ┆ 1102       ┆ … ┆ P060101005 ┆ 1        ┆ 170    ┆ 二ツ橋店   │
│ 20190205  ┆ 1549324800 ┆ S14042   ┆ 1132       ┆ … ┆ P050301001 ┆ 1        ┆ 25     

---
> P-037: 商品データ（df_product）とカテゴリデータ（df_category）を内部結合し、商品データの全項目とカテゴリデータのカテゴリ小区分名（category_small_name）を10件表示せよ。

In [70]:
df_product
    .clone()
    .inner_join(
        df_category.clone().select([col("category_small_cd"), col("category_small_name")]),
        col("category_small_cd"),
        col("category_small_cd"),
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 7)
┌────────────┬───────────────┬──────────────┬──────────────┬────────────┬───────────┬──────────────┐
│ product_cd ┆ category_majo ┆ category_med ┆ category_sma ┆ unit_price ┆ unit_cost ┆ category_sma │
│ ---        ┆ r_cd          ┆ ium_cd       ┆ ll_cd        ┆ ---        ┆ ---       ┆ ll_name      │
│ str        ┆ ---           ┆ ---          ┆ ---          ┆ i64        ┆ i64       ┆ ---          │
│            ┆ str           ┆ str          ┆ str          ┆            ┆           ┆ str          │
╞════════════╪═══════════════╪══════════════╪══════════════╪════════════╪═══════════╪══════════════╡
│ P040101001 ┆ 04            ┆ 0401         ┆ 040101       ┆ 198        ┆ 149       ┆ 弁当類       │
│ P040101002 ┆ 04            ┆ 0401         ┆ 040101       ┆ 218        ┆ 164       ┆ 弁当類       │
│ P040101003 ┆ 04            ┆ 0401         ┆ 040101       ┆ 230        ┆ 173       ┆ 弁当類       │
│ P040101004 ┆ 04            ┆ 0401         ┆ 040101       ┆ 248        ┆ 186       ┆

---
> P-038: 顧客データ（df_customer）とレシート明細データ（df_receipt）から、顧客ごとの売上金額合計を求め、10件表示せよ。ただし、売上実績がない顧客については売上金額を0として表示させること。また、顧客は性別コード（gender_cd）が女性（1）であるものを対象とし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [71]:
df_customer
    .clone()
    .filter(
        col("gender_cd").eq(lit("1"))
        .and(
            col("customer_id").str().starts_with(lit("Z")).not()
        )
    )
    .inner_join(
        df_receipt.clone(),
        col("customer_id"),
        col("customer_id"),
    )
    .groupby(["customer_id"])
    .agg([col("amount").sum().fill_null(0)])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 2)
┌────────────────┬────────┐
│ customer_id    ┆ amount │
│ ---            ┆ ---    │
│ str            ┆ i64    │
╞════════════════╪════════╡
│ CS008515000062 ┆ 1972   │
│ CS047211000001 ┆ 2293   │
│ CS013415000077 ┆ 5164   │
│ CS002515000266 ┆ 784    │
│ …              ┆ …      │
│ CS016315000115 ┆ 426    │
│ CS034415000038 ┆ 5039   │
│ CS016415000250 ┆ 7076   │
│ CS013614000024 ┆ 689    │
└────────────────┴────────┘

---
> P-039: レシート明細データ（df_receipt）から、売上日数の多い顧客の上位20件を抽出したデータと、売上金額合計の多い顧客の上位20件を抽出したデータをそれぞれ作成し、さらにその2つを完全外部結合せよ。ただし、非会員（顧客IDが"Z"から始まるもの）は除外すること。

In [72]:
let df_data = df_receipt.clone().filter(col("customer_id").str().starts_with(lit("Z")).not());

let df_cnt: LazyFrame = df_data.clone()
    .groupby(["customer_id"]).agg([col("sales_ymd").n_unique()])
    .sort("sales_ymd", SortOptions{descending: true, nulls_last: true, multithreaded: true, maintain_order: true})
    .slice(0, 20);
let df_sum: LazyFrame = df_data.clone().groupby(["customer_id"]).agg([col("amount").sum()])
    .sort("amount", SortOptions{descending: true, nulls_last: true, multithreaded: true, maintain_order: true})
    .slice(0, 20);
    
df_cnt
    .outer_join(
        df_sum.clone(),
        col("customer_id"),
        col("customer_id"),
    )
    .sort("amount", SortOptions{descending: false, nulls_last: true, multithreaded: true, maintain_order: true})
    .sort("sales_ymd", SortOptions{descending: true, nulls_last: true, multithreaded: true, maintain_order: true})
    .collect()
    .unwrap()

shape: (34, 3)
┌────────────────┬───────────┬────────┐
│ customer_id    ┆ sales_ymd ┆ amount │
│ ---            ┆ ---       ┆ ---    │
│ str            ┆ u32       ┆ i64    │
╞════════════════╪═══════════╪════════╡
│ CS040214000008 ┆ 23        ┆ null   │
│ CS010214000010 ┆ 22        ┆ 18585  │
│ CS015415000185 ┆ 22        ┆ 20153  │
│ CS028415000007 ┆ 21        ┆ 19127  │
│ …              ┆ …         ┆ …      │
│ CS007514000094 ┆ null      ┆ 15735  │
│ CS009414000059 ┆ null      ┆ 15492  │
│ CS030415000034 ┆ null      ┆ 15468  │
│ CS015515000034 ┆ null      ┆ 15300  │
└────────────────┴───────────┴────────┘

---
> P-040: 全ての店舗と全ての商品を組み合わせたデータを作成したい。店舗データ（df_store）と商品データ（df_product）を直積し、件数を計算せよ。

In [73]:
let df_store_tmp: LazyFrame = df_store.clone().with_columns([lit(0).alias("key")]);
let df_product_tmp: LazyFrame = df_product.clone().with_columns([lit(0).alias("key")]);

df_store_tmp.outer_join(df_product_tmp, col("key"), col("key"))
    .collect()
    .unwrap()
    .height()

531590

---
> P-041: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、前回売上があった日からの売上金額増減を計算せよ。そして結果を10件表示せよ。

In [74]:
df_receipt.clone()
    .groupby(["sales_ymd"]).agg([col("amount").sum()])
    .sort("sales_ymd", SortOptions{descending: false, nulls_last: true, multithreaded: true, maintain_order: true})
    .with_columns(
        [
            col("amount").shift(1).alias("-1day_amount"),
            (col("amount") - col("amount").shift(1)).alias("diff_amount"),
        ]
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 4)
┌───────────┬────────┬──────────────┬─────────────┐
│ sales_ymd ┆ amount ┆ -1day_amount ┆ diff_amount │
│ ---       ┆ ---    ┆ ---          ┆ ---         │
│ i64       ┆ i64    ┆ i64          ┆ i64         │
╞═══════════╪════════╪══════════════╪═════════════╡
│ 20170101  ┆ 33723  ┆ null         ┆ null        │
│ 20170102  ┆ 24165  ┆ 33723        ┆ -9558       │
│ 20170103  ┆ 27503  ┆ 24165        ┆ 3338        │
│ 20170104  ┆ 36165  ┆ 27503        ┆ 8662        │
│ …         ┆ …      ┆ …            ┆ …           │
│ 20170107  ┆ 23415  ┆ 32387        ┆ -8972       │
│ 20170108  ┆ 24737  ┆ 23415        ┆ 1322        │
│ 20170109  ┆ 26718  ┆ 24737        ┆ 1981        │
│ 20170110  ┆ 20143  ┆ 26718        ┆ -6575       │
└───────────┴────────┴──────────────┴─────────────┘

---
> P-042: レシート明細データ（df_receipt）の売上金額（amount）を日付（sales_ymd）ごとに集計し、各日付のデータに対し、前回、前々回、3回前に売上があった日のデータを結合せよ。そして結果を10件表示せよ。

In [75]:
df_receipt.clone()
    .groupby(["sales_ymd"]).agg([col("amount").sum()])
    .sort("sales_ymd", SortOptions{descending: false, nulls_last: true, multithreaded: true, maintain_order: true})
    .with_columns(
        [
            col("sales_ymd").shift(1).alias("1day_before_ymd"),
            col("amount").shift(1).alias("1day_before_amount"),
            col("sales_ymd").shift(2).alias("2day_before_ymd"),
            col("amount").shift(2).alias("2day_before_amount"),
            col("sales_ymd").shift(3).alias("3day_before_ymd"),
            col("amount").shift(3).alias("3day_before_amount"),
        ]
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 8)
┌───────────┬────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┐
│ sales_ymd ┆ amount ┆ 1day_befor ┆ 1day_befor ┆ 2day_befor ┆ 2day_befor ┆ 3day_befor ┆ 3day_befor │
│ ---       ┆ ---    ┆ e_ymd      ┆ e_amount   ┆ e_ymd      ┆ e_amount   ┆ e_ymd      ┆ e_amount   │
│ i64       ┆ i64    ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        │
│           ┆        ┆ i64        ┆ i64        ┆ i64        ┆ i64        ┆ i64        ┆ i64        │
╞═══════════╪════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╡
│ 20170101  ┆ 33723  ┆ null       ┆ null       ┆ null       ┆ null       ┆ null       ┆ null       │
│ 20170102  ┆ 24165  ┆ 20170101   ┆ 33723      ┆ null       ┆ null       ┆ null       ┆ null       │
│ 20170103  ┆ 27503  ┆ 20170102   ┆ 24165      ┆ 20170101   ┆ 33723      ┆ null       ┆ null       │
│ 20170104  ┆ 36165  ┆ 20170103   ┆ 27503      ┆ 20170102   ┆ 24165      ┆ 2

---
> P-043： レシート明細データ（df_receipt）と顧客データ（df_customer）を結合し、性別コード（gender_cd）と年代（ageから計算）ごとに売上金額（amount）を合計した売上サマリデータを作成せよ。性別コードは0が男性、1が女性、9が不明を表すものとする。
>
> ただし、項目構成は年代、女性の売上金額、男性の売上金額、性別不明の売上金額の4項目とすること（縦に年代、横に性別のクロス集計）。また、年代は10歳ごとの階級とすること。

In [76]:
let temp_df = df_receipt.clone()
    .inner_join(df_customer.clone(), col("customer_id"), col("customer_id")) 
    .with_column(
        ((col("age") / lit(10.0)).floor() * lit(10.0)).cast(DataType::Int32).alias("era"),
    )
    .groupby(["gender_cd", "era"]).agg([col("amount").sum()]);

let df_sales_summary = temp_df.clone()
    .select([col("era").unique()])
    .outer_join(
        temp_df.clone()
            .filter(col("gender_cd").eq(lit("0")))
            .select([col("era"), col("amount").alias("male")]),
        col("era"), 
        col("era"),
    )
    .outer_join(
        temp_df.clone()
            .filter(col("gender_cd").eq(lit("1")))
            .select([col("era"), col("amount").alias("female")]),
        col("era"), 
        col("era"),
    )
    .outer_join(
        temp_df.clone()
            .filter(col("gender_cd").eq(lit("9")))
            .select([col("era"), col("amount").alias("unknown")]),
        col("era"), 
        col("era"),
    )
    .sort("era", SortOptions{descending: false, nulls_last: true, multithreaded: true, maintain_order: true});

df_sales_summary.clone()
    .collect()
    .unwrap()

shape: (9, 4)
┌─────┬────────┬─────────┬─────────┐
│ era ┆ male   ┆ female  ┆ unknown │
│ --- ┆ ---    ┆ ---     ┆ ---     │
│ i32 ┆ i64    ┆ i64     ┆ i64     │
╞═════╪════════╪═════════╪═════════╡
│ 10  ┆ 1591   ┆ 149836  ┆ 4317    │
│ 20  ┆ 72940  ┆ 1363724 ┆ 44328   │
│ 30  ┆ 177322 ┆ 693047  ┆ 50441   │
│ 40  ┆ 19355  ┆ 9320791 ┆ 483512  │
│ 50  ┆ 54320  ┆ 6685192 ┆ 342923  │
│ 60  ┆ 272469 ┆ 987741  ┆ 71418   │
│ 70  ┆ 13435  ┆ 29764   ┆ 2427    │
│ 80  ┆ 46360  ┆ 262923  ┆ 5111    │
│ 90  ┆ null   ┆ 6260    ┆ null    │
└─────┴────────┴─────────┴─────────┘

---
> P-044： 043で作成した売上サマリデータ（df_sales_summary）は性別の売上を横持ちさせたものであった。このデータから性別を縦持ちさせ、年代、性別コード、売上金額の3項目に変換せよ。ただし、性別コードは男性を"00"、女性を"01"、不明を"99"とする。

In [77]:
fn conv_gender2cd(str_val: Series) -> Result<Option<Series>, PolarsError> {
    let x: ChunkedArray<Utf8Type> = str_val
        .utf8()
        .unwrap()
        .into_iter()
        // your actual custom function would be in this map
        .map(
            |opt_name: Option<&str>| opt_name.map(
                |name: &str| match name {
                    "male" => "00",
                    "female" => "01",
                    "unknown" => "99",
                    _ => "",
                }
            )
        )
        .collect();
    Ok(Some(x.into_series()))
}

let melt_args: MeltArgs = MeltArgs {
    id_vars: vec!["era".into()],
    value_vars: vec!["male".into(), "female".into(), "unknown".into()],
    variable_name: Some("gender".into()),
    value_name: Some("amount".into()),
    streamable: true,
};

let melt_df = df_sales_summary.clone().melt(melt_args);


melt_df.clone()
    .select([
        col("era"),
        col("gender")
        .apply(
            |x| conv_gender2cd(x),
            GetOutput::from_type(DataType::UInt32)
        ).alias("gender_cd"),
        col("amount"),
    ])
    .sort(
        "era",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    )
    .collect()
    .unwrap()

shape: (27, 3)
┌─────┬───────────┬────────┐
│ era ┆ gender_cd ┆ amount │
│ --- ┆ ---       ┆ ---    │
│ i32 ┆ str       ┆ i64    │
╞═════╪═══════════╪════════╡
│ 10  ┆ 00        ┆ 1591   │
│ 10  ┆ 01        ┆ 149836 │
│ 10  ┆ 99        ┆ 4317   │
│ 20  ┆ 00        ┆ 72940  │
│ …   ┆ …         ┆ …      │
│ 80  ┆ 99        ┆ 5111   │
│ 90  ┆ 00        ┆ null   │
│ 90  ┆ 01        ┆ 6260   │
│ 90  ┆ 99        ┆ null   │
└─────┴───────────┴────────┘

In [142]:
// when().then().otherwise()を使用するほうがスッキリかける

let melt_args: MeltArgs = MeltArgs {
    id_vars: vec!["era".into()],
    value_vars: vec!["male".into(), "female".into(), "unknown".into()],
    variable_name: Some("gender".into()),
    value_name: Some("amount".into()),
    streamable: true,
};

let melt_df = df_sales_summary.clone().melt(melt_args);

melt_df.clone()
    .select([
        col("era"),
        when(col("gender").eq(lit("male"))).then(lit("00"))
            .when(col("gender").eq(lit("female"))).then(lit("01"))
            .otherwise(lit("99"))
            .alias("gender_cd"),
        col("amount"),
    ])
    .sort(
        "era",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    )
    .collect()
    .unwrap()

shape: (27, 3)
┌─────┬───────────┬────────┐
│ era ┆ gender_cd ┆ amount │
│ --- ┆ ---       ┆ ---    │
│ i32 ┆ str       ┆ i64    │
╞═════╪═══════════╪════════╡
│ 10  ┆ 00        ┆ 1591   │
│ 10  ┆ 01        ┆ 149836 │
│ 10  ┆ 99        ┆ 4317   │
│ 20  ┆ 00        ┆ 72940  │
│ …   ┆ …         ┆ …      │
│ 80  ┆ 99        ┆ 5111   │
│ 90  ┆ 00        ┆ null   │
│ 90  ┆ 01        ┆ 6260   │
│ 90  ┆ 99        ┆ null   │
└─────┴───────────┴────────┘

---
> P-045: 顧客データ（df_customer）の生年月日（birth_day）は日付型でデータを保有している。これをYYYYMMDD形式の文字列に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [78]:
df_customer.clone()
    .select([
        col("customer_id"),
        col("birth_day").dt().to_string("%Y%m%d"),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 2)
┌────────────────┬───────────┐
│ customer_id    ┆ birth_day │
│ ---            ┆ ---       │
│ str            ┆ str       │
╞════════════════╪═══════════╡
│ CS021313000114 ┆ 19810429  │
│ CS037613000071 ┆ 19520401  │
│ CS031415000172 ┆ 19761004  │
│ CS028811000001 ┆ 19330327  │
│ …              ┆ …         │
│ CS015414000103 ┆ 19770809  │
│ CS029403000008 ┆ 19730817  │
│ CS015804000004 ┆ 19310502  │
│ CS033513000180 ┆ 19620711  │
└────────────────┴───────────┘

---
> P-046: 顧客データ（df_customer）の申し込み日（application_date）はYYYYMMDD形式の文字列型でデータを保有している。これを日付型に変換し、顧客ID（customer_id）とともに10件表示せよ。

In [94]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%Y%m%d".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_customer.clone()
    .select([
        col("customer_id"),
        col("application_date").str().to_date(options),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 2)
┌────────────────┬──────────────────┐
│ customer_id    ┆ application_date │
│ ---            ┆ ---              │
│ str            ┆ date             │
╞════════════════╪══════════════════╡
│ CS021313000114 ┆ 2015-09-05       │
│ CS037613000071 ┆ 2015-04-14       │
│ CS031415000172 ┆ 2015-05-29       │
│ CS028811000001 ┆ 2016-01-15       │
│ …              ┆ …                │
│ CS015414000103 ┆ 2015-07-22       │
│ CS029403000008 ┆ 2015-05-15       │
│ CS015804000004 ┆ 2015-06-07       │
│ CS033513000180 ┆ 2015-07-28       │
└────────────────┴──────────────────┘

---
> P-047: レシート明細データ（df_receipt）の売上日（sales_ymd）はYYYYMMDD形式の数値型でデータを保有している。これを日付型に変換し、レシート番号（receipt_no）、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [125]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%Y%m%d".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .select([
        col("receipt_no"),
        col("receipt_sub_no"),
        col("sales_ymd").cast(DataType::Utf8).str().to_date(options),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────┬────────────────┬────────────┐
│ receipt_no ┆ receipt_sub_no ┆ sales_ymd  │
│ ---        ┆ ---            ┆ ---        │
│ i64        ┆ i64            ┆ date       │
╞════════════╪════════════════╪════════════╡
│ 112        ┆ 1              ┆ 2018-11-03 │
│ 1132       ┆ 2              ┆ 2018-11-18 │
│ 1102       ┆ 1              ┆ 2017-07-12 │
│ 1132       ┆ 1              ┆ 2019-02-05 │
│ …          ┆ …              ┆ …          │
│ 1102       ┆ 2              ┆ 2018-12-05 │
│ 1102       ┆ 1              ┆ 2019-09-22 │
│ 1112       ┆ 2              ┆ 2017-05-04 │
│ 1102       ┆ 1              ┆ 2019-10-10 │
└────────────┴────────────────┴────────────┘

---
> P-048: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）は数値型のUNIX秒でデータを保有している。これを日付型に変換し、レシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [126]:
// to_datetimeを使用しない場合
// df_receipt.clone()
//     .select([
//         col("receipt_no"),
//         col("receipt_sub_no"),
//         (col("sales_epoch") * lit(1000))
//             .cast(DataType::Datetime(
//                 TimeUnit::Milliseconds,
//                 Some("Tokyo".into())
//             ))
//             .dt().date()
//             .alias("sales_ymd"),
//     ])
//     .collect()
//     .unwrap()
//     .head(Some(10))

let options: StrptimeOptions = StrptimeOptions {
    format: Some("%s".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .select([
        col("receipt_no"),
        col("receipt_sub_no"),
        col("sales_epoch").cast(DataType::Utf8)
            .str().to_datetime(
                Some(TimeUnit::Milliseconds),
                Some("Tokyo".into()),
                options,
            )
            .dt().date()
            .alias("sales_ymd"),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────┬────────────────┬────────────┐
│ receipt_no ┆ receipt_sub_no ┆ sales_ymd  │
│ ---        ┆ ---            ┆ ---        │
│ i64        ┆ i64            ┆ date       │
╞════════════╪════════════════╪════════════╡
│ 112        ┆ 1              ┆ 2018-11-03 │
│ 1132       ┆ 2              ┆ 2018-11-18 │
│ 1102       ┆ 1              ┆ 2017-07-12 │
│ 1132       ┆ 1              ┆ 2019-02-05 │
│ …          ┆ …              ┆ …          │
│ 1102       ┆ 2              ┆ 2018-12-05 │
│ 1102       ┆ 1              ┆ 2019-09-22 │
│ 1112       ┆ 2              ┆ 2017-05-04 │
│ 1102       ┆ 1              ┆ 2019-10-10 │
└────────────┴────────────────┴────────────┘

---
> P-049: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「年」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。

In [127]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%s".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .select([
        col("receipt_no"),
        col("receipt_sub_no"),
        col("sales_epoch").cast(DataType::Utf8)
            .str().to_datetime(
                Some(TimeUnit::Milliseconds),
                Some("Tokyo".into()),
                options,
            )
            .dt().year()
            .alias("sales_year"),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────┬────────────────┬────────────┐
│ receipt_no ┆ receipt_sub_no ┆ sales_year │
│ ---        ┆ ---            ┆ ---        │
│ i64        ┆ i64            ┆ i32        │
╞════════════╪════════════════╪════════════╡
│ 112        ┆ 1              ┆ 2018       │
│ 1132       ┆ 2              ┆ 2018       │
│ 1102       ┆ 1              ┆ 2017       │
│ 1132       ┆ 1              ┆ 2019       │
│ …          ┆ …              ┆ …          │
│ 1102       ┆ 2              ┆ 2018       │
│ 1102       ┆ 1              ┆ 2019       │
│ 1112       ┆ 2              ┆ 2017       │
│ 1102       ┆ 1              ┆ 2019       │
└────────────┴────────────────┴────────────┘

---
> P-050: レシート明細データ（df_receipt）の売上エポック秒（sales_epoch）を日付型に変換し、「月」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「月」は0埋め2桁で取り出すこと。

In [128]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%s".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .select([
        col("receipt_no"),
        col("receipt_sub_no"),
        col("sales_epoch").cast(DataType::Utf8)
            .str().to_datetime(
                Some(TimeUnit::Milliseconds),
                Some("Tokyo".into()),
                options,
            )
            .dt().month()
            .alias("sales_month"),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────┬────────────────┬─────────────┐
│ receipt_no ┆ receipt_sub_no ┆ sales_month │
│ ---        ┆ ---            ┆ ---         │
│ i64        ┆ i64            ┆ u32         │
╞════════════╪════════════════╪═════════════╡
│ 112        ┆ 1              ┆ 11          │
│ 1132       ┆ 2              ┆ 11          │
│ 1102       ┆ 1              ┆ 7           │
│ 1132       ┆ 1              ┆ 2           │
│ …          ┆ …              ┆ …           │
│ 1102       ┆ 2              ┆ 12          │
│ 1102       ┆ 1              ┆ 9           │
│ 1112       ┆ 2              ┆ 5           │
│ 1102       ┆ 1              ┆ 10          │
└────────────┴────────────────┴─────────────┘

---
> P-051: レシート明細データ（df_receipt）の売上エポック秒を日付型に変換し、「日」だけ取り出してレシート番号(receipt_no)、レシートサブ番号（receipt_sub_no）とともに10件表示せよ。なお、「日」は0埋め2桁で取り出すこと。

In [130]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%s".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .select([
        col("receipt_no"),
        col("receipt_sub_no"),
        col("sales_epoch").cast(DataType::Utf8)
            .str().to_datetime(
                Some(TimeUnit::Milliseconds),
                Some("Tokyo".into()),
                options,
            )
            .dt().to_string("%d".into())
            .alias("sales_day"),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────┬────────────────┬──────────────┐
│ receipt_no ┆ receipt_sub_no ┆ sales_day    │
│ ---        ┆ ---            ┆              │
│ i64        ┆ i64            ┆          …   │
│            ┆                ┆ ---          │
│            ┆                ┆ str          │
╞════════════╪════════════════╪══════════════╡
│ 112        ┆ 1              ┆ 03           │
│ 1132       ┆ 2              ┆ 18           │
│ 1102       ┆ 1              ┆ 12           │
│ 1132       ┆ 1              ┆ 05           │
│ …          ┆ …              ┆ …            │
│ 1102       ┆ 2              ┆ 05           │
│ 1102       ┆ 1              ┆ 22           │
│ 1112       ┆ 2              ┆ 04           │
│ 1102       ┆ 1              ┆ 10           │
└────────────┴────────────────┴──────────────┘

---
> P-052: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計の上、売上金額合計に対して2,000円以下を0、2,000円より大きい金額を1に二値化し、顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [143]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%s".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .filter(col("customer_id").str().starts_with(lit("Z")).not())
    .groupby(["customer_id"]).agg([col("amount").sum()])
    .select([
        col("customer_id"),
        col("amount"),
        when(col("amount").gt_eq(lit(2000)))
            .then(lit(1))
            .otherwise(lit(0))
            .alias("sales_flg")
    ])
    .sort(
        "customer_id",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────────┬────────┬───────────┐
│ customer_id    ┆ amount ┆ sales_flg │
│ ---            ┆ ---    ┆ ---       │
│ str            ┆ i64    ┆ i32       │
╞════════════════╪════════╪═══════════╡
│ CS001113000004 ┆ 1298   ┆ 0         │
│ CS001114000005 ┆ 626    ┆ 0         │
│ CS001115000010 ┆ 3044   ┆ 1         │
│ CS001205000004 ┆ 1988   ┆ 0         │
│ …              ┆ …      ┆ …         │
│ CS001212000027 ┆ 448    ┆ 0         │
│ CS001212000031 ┆ 296    ┆ 0         │
│ CS001212000046 ┆ 228    ┆ 0         │
│ CS001212000070 ┆ 456    ┆ 0         │
└────────────────┴────────┴───────────┘

---
> P-053: 顧客データ（df_customer）の郵便番号（postal_cd）に対し、東京（先頭3桁が100〜209のもの）を1、それ以外のものを0に二値化せよ。さらにレシート明細データ（df_receipt）と結合し、全期間において売上実績のある顧客数を、作成した二値ごとにカウントせよ。

In [170]:
df_customer.clone()
    .with_column(col("postal_cd").str().str_slice(0, Some(3)).cast(DataType::Int16).alias("postal_flg"))
    .select([
        col("customer_id"),
        when(col("postal_flg").gt_eq(lit(100)).and(col("postal_flg").lt_eq(lit(209))))
            .then(lit(1))
            .otherwise(lit(0)).alias("postal_flg"),
    ])
    .inner_join(df_receipt.clone(), col("customer_id"), col("customer_id"))
    .groupby(["postal_flg"]).agg([col("customer_id").n_unique()])
    .collect()
    .unwrap()


shape: (2, 2)
┌────────────┬─────────────┐
│ postal_flg ┆ customer_id │
│ ---        ┆ ---         │
│ i32        ┆ u32         │
╞════════════╪═════════════╡
│ 0          ┆ 3906        │
│ 1          ┆ 4400        │
└────────────┴─────────────┘

---
> P-054: 顧客データ（df_customer）の住所（address）は、埼玉県、千葉県、東京都、神奈川県のいずれかとなっている。都道府県毎にコード値を作成し、顧客ID、住所とともに10件表示せよ。値は埼玉県を11、千葉県を12、東京都を13、神奈川県を14とすること。

In [177]:
df_customer.clone()
    .with_column(
        when(col("address").str().contains(lit("埼玉県"), true)).then(lit(11))
        .when(col("address").str().contains(lit("千葉県"), true)).then(lit(12))
        .when(col("address").str().contains(lit("東京都"), true)).then(lit(13))
        .when(col("address").str().contains(lit("神奈川県"), true)).then(lit(14))
        .otherwise(lit("")).alias("prefecture_cd")
    ).select([
        col("customer_id"),
        col("address"),
        col("prefecture_cd"),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────────┬───────────────────────────────────┬───────────────┐
│ customer_id    ┆ address                           ┆ prefecture_cd │
│ ---            ┆ ---                               ┆ ---           │
│ str            ┆ str                               ┆ str           │
╞════════════════╪═══════════════════════════════════╪═══════════════╡
│ CS021313000114 ┆ 神奈川県伊勢原市粟窪**********    ┆ 14            │
│ CS037613000071 ┆ 東京都江東区南砂**********        ┆ 13            │
│ CS031415000172 ┆ 東京都渋谷区代々木**********      ┆ 13            │
│ CS028811000001 ┆ 神奈川県横浜市泉区和泉町********* ┆ 14            │
│                ┆ *                                 ┆               │
│ …              ┆ …                                 ┆ …             │
│ CS015414000103 ┆ 東京都江東区北砂**********        ┆ 13            │
│ CS029403000008 ┆ 千葉県浦安市海楽**********        ┆ 12            │
│ CS015804000004 ┆ 東京都江東区北砂**********        ┆ 13            │
│ CS033513000180 ┆ 神奈川県横浜市旭区善部町********* ┆ 14         

---
> P-055: レシート明細（df_receipt）データの売上金額（amount）を顧客ID（customer_id）ごとに合計し、その合計金額の四分位点を求めよ。その上で、顧客ごとの売上金額合計に対して以下の基準でカテゴリ値を作成し、顧客ID、売上金額合計とともに10件表示せよ。カテゴリ値は順に1〜4とする。
>
> - 最小値以上第1四分位未満 ・・・ 1を付与
> - 第1四分位以上第2四分位未満 ・・・ 2を付与
> - 第2四分位以上第3四分位未満 ・・・ 3を付与
> - 第3四分位以上 ・・・ 4を付与

In [181]:
df_receipt.clone()
    .groupby(["customer_id"]).agg([col("amount").sum()])
    .with_column(
        when(col("amount").lt(col("amount").quantile(lit(0.25), QuantileInterpolOptions::Nearest))).then(lit(1))
        .when(col("amount").lt(col("amount").quantile(lit(0.50), QuantileInterpolOptions::Nearest))).then(lit(2))
        .when(col("amount").lt(col("amount").quantile(lit(0.75), QuantileInterpolOptions::Nearest))).then(lit(3))
        .otherwise(lit(4)).alias("pct_group")
    ).sort(
        "customer_id",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    ).collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────────┬────────┬───────────┐
│ customer_id    ┆ amount ┆ pct_group │
│ ---            ┆ ---    ┆ ---       │
│ str            ┆ i64    ┆ i32       │
╞════════════════╪════════╪═══════════╡
│ CS001113000004 ┆ 1298   ┆ 2         │
│ CS001114000005 ┆ 626    ┆ 2         │
│ CS001115000010 ┆ 3044   ┆ 3         │
│ CS001205000004 ┆ 1988   ┆ 3         │
│ …              ┆ …      ┆ …         │
│ CS001212000027 ┆ 448    ┆ 1         │
│ CS001212000031 ┆ 296    ┆ 1         │
│ CS001212000046 ┆ 228    ┆ 1         │
│ CS001212000070 ┆ 456    ┆ 1         │
└────────────────┴────────┴───────────┘

---
> P-056: 顧客データ（df_customer）の年齢（age）をもとに10歳刻みで年代を算出し、顧客ID（customer_id）、生年月日（birth_day）とともに10件表示せよ。ただし、60歳以上は全て60歳代とすること。年代を表すカテゴリ名は任意とする。

In [220]:
df_customer.clone()
    .select([
        col("customer_id"),
        col("birth_day"),
        when(col("age").gt_eq(lit(60))).then(60)
            .otherwise(((col("age") / lit(10.0)).floor() * lit(10.0)).cast(DataType::Int32))
            .alias("era"),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────────┬────────────┬─────┐
│ customer_id    ┆ birth_day  ┆ era │
│ ---            ┆ ---        ┆ --- │
│ str            ┆ date       ┆ i32 │
╞════════════════╪════════════╪═════╡
│ CS021313000114 ┆ 1981-04-29 ┆ 30  │
│ CS037613000071 ┆ 1952-04-01 ┆ 60  │
│ CS031415000172 ┆ 1976-10-04 ┆ 40  │
│ CS028811000001 ┆ 1933-03-27 ┆ 60  │
│ …              ┆ …          ┆ …   │
│ CS015414000103 ┆ 1977-08-09 ┆ 40  │
│ CS029403000008 ┆ 1973-08-17 ┆ 40  │
│ CS015804000004 ┆ 1931-05-02 ┆ 60  │
│ CS033513000180 ┆ 1962-07-11 ┆ 50  │
└────────────────┴────────────┴─────┘

---
> P-057: 056の抽出結果と性別コード（gender_cd）により、新たに性別×年代の組み合わせを表すカテゴリデータを作成し、10件表示せよ。組み合わせを表すカテゴリの値は任意とする。

In [4]:
df_customer.clone()
    .with_column(
        when(col("age").gt_eq(lit(60))).then(60)
            .otherwise(((col("age") / lit(10.0)).floor() * lit(10.0)).cast(DataType::Int32))
            .alias("era"),
    )
    .select([
        col("customer_id"),
        col("birth_day"),
        col("era"),
        concat_str([col("gender_cd"), col("era").cast(DataType::Utf8)], ""),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 4)
┌────────────────┬────────────┬─────┬───────────┐
│ customer_id    ┆ birth_day  ┆ era ┆ gender_cd │
│ ---            ┆ ---        ┆ --- ┆ ---       │
│ str            ┆ date       ┆ i32 ┆ str       │
╞════════════════╪════════════╪═════╪═══════════╡
│ CS021313000114 ┆ 1981-04-29 ┆ 30  ┆ 130       │
│ CS037613000071 ┆ 1952-04-01 ┆ 60  ┆ 960       │
│ CS031415000172 ┆ 1976-10-04 ┆ 40  ┆ 140       │
│ CS028811000001 ┆ 1933-03-27 ┆ 60  ┆ 160       │
│ …              ┆ …          ┆ …   ┆ …         │
│ CS015414000103 ┆ 1977-08-09 ┆ 40  ┆ 140       │
│ CS029403000008 ┆ 1973-08-17 ┆ 40  ┆ 040       │
│ CS015804000004 ┆ 1931-05-02 ┆ 60  ┆ 060       │
│ CS033513000180 ┆ 1962-07-11 ┆ 50  ┆ 150       │
└────────────────┴────────────┴─────┴───────────┘

---
> P-058: 顧客データ（df_customer）の性別コード（gender_cd）をダミー変数化し、顧客ID（customer_id）とともに10件表示せよ。

In [8]:
df_customer.clone()
    .select([
        col("customer_id"),
        col("gender_cd"),
    ])
    .collect()
    .unwrap()
    .columns_to_dummies(vec!["gender_cd"], None, false)
    .unwrap()
    .head(Some(10))

shape: (10, 4)
┌────────────────┬─────────────┬─────────────┬─────────────┐
│ customer_id    ┆ gender_cd_0 ┆ gender_cd_1 ┆ gender_cd_9 │
│ ---            ┆ ---         ┆ ---         ┆ ---         │
│ str            ┆ u8          ┆ u8          ┆ u8          │
╞════════════════╪═════════════╪═════════════╪═════════════╡
│ CS021313000114 ┆ 0           ┆ 1           ┆ 0           │
│ CS037613000071 ┆ 0           ┆ 0           ┆ 1           │
│ CS031415000172 ┆ 0           ┆ 1           ┆ 0           │
│ CS028811000001 ┆ 0           ┆ 1           ┆ 0           │
│ …              ┆ …           ┆ …           ┆ …           │
│ CS015414000103 ┆ 0           ┆ 1           ┆ 0           │
│ CS029403000008 ┆ 1           ┆ 0           ┆ 0           │
│ CS015804000004 ┆ 1           ┆ 0           ┆ 0           │
│ CS033513000180 ┆ 0           ┆ 1           ┆ 0           │
└────────────────┴─────────────┴─────────────┴─────────────┘

---
> P-059: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を平均0、標準偏差1に標準化して顧客ID、売上金額合計とともに10件表示せよ。標準化に使用する標準偏差は、分散の平方根、もしくは不偏分散の平方根のどちらでも良いものとする。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

TIPS:
- query()の引数engineで"python"か"numexpr"かを選択でき、デフォルトはインストールされていればnumexprが、無ければpythonが使われます。さらに、文字列メソッドはengine="python"でないとquery()内で使えません。


In [19]:
let amount: Expr = col("amount");

df_receipt.clone()
    .filter(
        col("customer_id").str().starts_with(lit("Z")).not()
    )
    .groupby(["customer_id"]).agg([col("amount").sum()])
    .select([
        col("customer_id"),
        col("amount"),
        ((amount.clone() - amount.clone().mean()) / amount.clone().std(0)).alias("std_amount")
    ])
    .sort(
        "customer_id",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────────┬────────┬────────────┐
│ customer_id    ┆ amount ┆ std_amount │
│ ---            ┆ ---    ┆ ---        │
│ str            ┆ i64    ┆ f64        │
╞════════════════╪════════╪════════════╡
│ CS001113000004 ┆ 1298   ┆ -0.459378  │
│ CS001114000005 ┆ 626    ┆ -0.70639   │
│ CS001115000010 ┆ 3044   ┆ 0.182413   │
│ CS001205000004 ┆ 1988   ┆ -0.205749  │
│ …              ┆ …      ┆ …          │
│ CS001212000027 ┆ 448    ┆ -0.771819  │
│ CS001212000031 ┆ 296    ┆ -0.827691  │
│ CS001212000046 ┆ 228    ┆ -0.852686  │
│ CS001212000070 ┆ 456    ┆ -0.768879  │
└────────────────┴────────┴────────────┘

---
> P-060: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を最小値0、最大値1に正規化して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [26]:
let amount: Expr = col("amount");

df_receipt.clone()
    .filter(col("customer_id").str().starts_with(lit("Z")).not())
    .groupby(["customer_id"]).agg([col("amount").sum()])
    .select([
        col("customer_id"),
        col("amount"),
        ((amount.clone() - amount.clone().min()).cast(DataType::Float64) / (amount.clone().max() - amount.clone().min())).alias("scale_amount")
    ])
    .sort(
        "customer_id",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────────┬────────┬──────────────┐
│ customer_id    ┆ amount ┆ scale_amount │
│ ---            ┆ ---    ┆ ---          │
│ str            ┆ i64    ┆ f64          │
╞════════════════╪════════╪══════════════╡
│ CS001113000004 ┆ 1298   ┆ 0.053354     │
│ CS001114000005 ┆ 626    ┆ 0.024157     │
│ CS001115000010 ┆ 3044   ┆ 0.129214     │
│ CS001205000004 ┆ 1988   ┆ 0.083333     │
│ …              ┆ …      ┆ …            │
│ CS001212000027 ┆ 448    ┆ 0.016423     │
│ CS001212000031 ┆ 296    ┆ 0.009819     │
│ CS001212000046 ┆ 228    ┆ 0.006865     │
│ CS001212000070 ┆ 456    ┆ 0.016771     │
└────────────────┴────────┴──────────────┘

---
> P-061: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を常用対数化（底10）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [37]:
df_receipt.clone()
    .filter(col("customer_id").str().starts_with(lit("Z")).not())
    .groupby(["customer_id"]).agg([col("amount").sum() + lit(0.5)]) // +0.5の理由?
    .with_column(col("amount").log(10.0).alias("log_amount"))
    .sort(
        "customer_id",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────────┬────────┬────────────┐
│ customer_id    ┆ amount ┆ log_amount │
│ ---            ┆ ---    ┆ ---        │
│ str            ┆ f64    ┆ f64        │
╞════════════════╪════════╪════════════╡
│ CS001113000004 ┆ 1298.5 ┆ 3.113442   │
│ CS001114000005 ┆ 626.5  ┆ 2.796921   │
│ CS001115000010 ┆ 3044.5 ┆ 3.483516   │
│ CS001205000004 ┆ 1988.5 ┆ 3.298526   │
│ …              ┆ …      ┆ …          │
│ CS001212000027 ┆ 448.5  ┆ 2.651762   │
│ CS001212000031 ┆ 296.5  ┆ 2.472025   │
│ CS001212000046 ┆ 228.5  ┆ 2.358886   │
│ CS001212000070 ┆ 456.5  ┆ 2.659441   │
└────────────────┴────────┴────────────┘

---
> P-062: レシート明細データ（df_receipt）の売上金額（amount）を顧客ID（customer_id）ごとに合計し、売上金額合計を自然対数化（底e）して顧客ID、売上金額合計とともに10件表示せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。

In [39]:
df_receipt.clone()
    .filter(col("customer_id").str().starts_with(lit("Z")).not())
    .groupby(["customer_id"]).agg([col("amount").sum() - lit(0.5)]) // -0.5の理由?
    .with_column(col("amount").log1p().alias("log_amount"))  // log1pはln(x + 1)を計算している
    .sort(
        "customer_id",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────────┬────────┬────────────┐
│ customer_id    ┆ amount ┆ log_amount │
│ ---            ┆ ---    ┆ ---        │
│ str            ┆ f64    ┆ f64        │
╞════════════════╪════════╪════════════╡
│ CS001113000004 ┆ 1297.5 ┆ 7.168965   │
│ CS001114000005 ┆ 625.5  ┆ 6.440149   │
│ CS001115000010 ┆ 3043.5 ┆ 8.021092   │
│ CS001205000004 ┆ 1987.5 ┆ 7.595136   │
│ …              ┆ …      ┆ …          │
│ CS001212000027 ┆ 447.5  ┆ 6.105909   │
│ CS001212000031 ┆ 295.5  ┆ 5.692047   │
│ CS001212000046 ┆ 227.5  ┆ 5.431536   │
│ CS001212000070 ┆ 455.5  ┆ 6.123589   │
└────────────────┴────────┴────────────┘

---
> P-063: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から各商品の利益額を算出し、結果を10件表示せよ。

In [40]:
df_product.clone()
    .with_column((col("unit_price") - col("unit_cost")).alias("unit_profit"))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 7)
┌────────────┬───────────────┬───────────────┬──────────────┬────────────┬───────────┬─────────────┐
│ product_cd ┆ category_majo ┆ category_medi ┆ category_sma ┆ unit_price ┆ unit_cost ┆ unit_profit │
│ ---        ┆ r_cd          ┆ um_cd         ┆ ll_cd        ┆ ---        ┆ ---       ┆ ---         │
│ str        ┆ ---           ┆ ---           ┆ ---          ┆ i64        ┆ i64       ┆ i64         │
│            ┆ str           ┆ str           ┆ str          ┆            ┆           ┆             │
╞════════════╪═══════════════╪═══════════════╪══════════════╪════════════╪═══════════╪═════════════╡
│ P040101001 ┆ 04            ┆ 0401          ┆ 040101       ┆ 198        ┆ 149       ┆ 49          │
│ P040101002 ┆ 04            ┆ 0401          ┆ 040101       ┆ 218        ┆ 164       ┆ 54          │
│ P040101003 ┆ 04            ┆ 0401          ┆ 040101       ┆ 230        ┆ 173       ┆ 57          │
│ P040101004 ┆ 04            ┆ 0401          ┆ 040101       ┆ 248        ┆ 1

---
> P-064: 商品データ（df_product）の単価（unit_price）と原価（unit_cost）から、各商品の利益率の全体平均を算出せよ。ただし、単価と原価には欠損が生じていることに注意せよ。

In [46]:
df_product.clone()
    .select([((col("unit_price") - col("unit_cost")).cast(DataType::Float64) / col("unit_price")).alias("unit_profit_rate")])
    .mean()
    .collect()
    .unwrap()

shape: (1, 1)
┌──────────────────┐
│ unit_profit_rate │
│ ---              │
│ f64              │
╞══════════════════╡
│ 0.249114         │
└──────────────────┘

---
> P-065: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。ただし、1円未満は切り捨てること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [58]:
df_product.clone()
    .select([
        col("product_cd"),
        col("unit_price"),
        col("unit_cost"),
        (col("unit_cost") / lit(0.7)).floor().alias("new_price"),
    ])
    .with_column(((col("new_price") - col("unit_cost")) / col("new_price")).alias("new_profit_rate"))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 5)
┌────────────┬────────────┬───────────┬───────────┬─────────────────┐
│ product_cd ┆ unit_price ┆ unit_cost ┆ new_price ┆ new_profit_rate │
│ ---        ┆ ---        ┆ ---       ┆ ---       ┆ ---             │
│ str        ┆ i64        ┆ i64       ┆ f64       ┆ f64             │
╞════════════╪════════════╪═══════════╪═══════════╪═════════════════╡
│ P040101001 ┆ 198        ┆ 149       ┆ 212.0     ┆ 0.29717         │
│ P040101002 ┆ 218        ┆ 164       ┆ 234.0     ┆ 0.299145        │
│ P040101003 ┆ 230        ┆ 173       ┆ 247.0     ┆ 0.299595        │
│ P040101004 ┆ 248        ┆ 186       ┆ 265.0     ┆ 0.298113        │
│ …          ┆ …          ┆ …         ┆ …         ┆ …               │
│ P040101007 ┆ 338        ┆ 254       ┆ 362.0     ┆ 0.298343        │
│ P040101008 ┆ 420        ┆ 315       ┆ 450.0     ┆ 0.3             │
│ P040101009 ┆ 498        ┆ 374       ┆ 534.0     ┆ 0.299625        │
│ P040101010 ┆ 580        ┆ 435       ┆ 621.0     ┆ 0.299517        │
└────

---
> P-066: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を丸めること（四捨五入または偶数への丸めで良い）。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [60]:
df_product.clone()
    .select([
        col("product_cd"),
        col("unit_price"),
        col("unit_cost"),
        (col("unit_cost") / lit(0.7)).round(0).alias("new_price"),
    ])
    .with_column(((col("new_price") - col("unit_cost")) / col("new_price")).alias("new_profit_rate"))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 5)
┌────────────┬────────────┬───────────┬───────────┬─────────────────┐
│ product_cd ┆ unit_price ┆ unit_cost ┆ new_price ┆ new_profit_rate │
│ ---        ┆ ---        ┆ ---       ┆ ---       ┆ ---             │
│ str        ┆ i64        ┆ i64       ┆ f64       ┆ f64             │
╞════════════╪════════════╪═══════════╪═══════════╪═════════════════╡
│ P040101001 ┆ 198        ┆ 149       ┆ 213.0     ┆ 0.300469        │
│ P040101002 ┆ 218        ┆ 164       ┆ 234.0     ┆ 0.299145        │
│ P040101003 ┆ 230        ┆ 173       ┆ 247.0     ┆ 0.299595        │
│ P040101004 ┆ 248        ┆ 186       ┆ 266.0     ┆ 0.300752        │
│ …          ┆ …          ┆ …         ┆ …         ┆ …               │
│ P040101007 ┆ 338        ┆ 254       ┆ 363.0     ┆ 0.300275        │
│ P040101008 ┆ 420        ┆ 315       ┆ 450.0     ┆ 0.3             │
│ P040101009 ┆ 498        ┆ 374       ┆ 534.0     ┆ 0.299625        │
│ P040101010 ┆ 580        ┆ 435       ┆ 621.0     ┆ 0.299517        │
└────

---
> P-067: 商品データ（df_product）の各商品について、利益率が30%となる新たな単価を求めよ。今回は、1円未満を切り上げること。そして結果を10件表示させ、利益率がおよそ30％付近であることを確認せよ。ただし、単価（unit_price）と原価（unit_cost）には欠損が生じていることに注意せよ。

In [61]:
df_product.clone()
    .select([
        col("product_cd"),
        col("unit_price"),
        col("unit_cost"),
        (col("unit_cost") / lit(0.7)).ceil().alias("new_price"),
    ])
    .with_column(((col("new_price") - col("unit_cost")) / col("new_price")).alias("new_profit_rate"))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 5)
┌────────────┬────────────┬───────────┬───────────┬─────────────────┐
│ product_cd ┆ unit_price ┆ unit_cost ┆ new_price ┆ new_profit_rate │
│ ---        ┆ ---        ┆ ---       ┆ ---       ┆ ---             │
│ str        ┆ i64        ┆ i64       ┆ f64       ┆ f64             │
╞════════════╪════════════╪═══════════╪═══════════╪═════════════════╡
│ P040101001 ┆ 198        ┆ 149       ┆ 213.0     ┆ 0.300469        │
│ P040101002 ┆ 218        ┆ 164       ┆ 235.0     ┆ 0.302128        │
│ P040101003 ┆ 230        ┆ 173       ┆ 248.0     ┆ 0.302419        │
│ P040101004 ┆ 248        ┆ 186       ┆ 266.0     ┆ 0.300752        │
│ …          ┆ …          ┆ …         ┆ …         ┆ …               │
│ P040101007 ┆ 338        ┆ 254       ┆ 363.0     ┆ 0.300275        │
│ P040101008 ┆ 420        ┆ 315       ┆ 451.0     ┆ 0.301552        │
│ P040101009 ┆ 498        ┆ 374       ┆ 535.0     ┆ 0.300935        │
│ P040101010 ┆ 580        ┆ 435       ┆ 622.0     ┆ 0.300643        │
└────

---
> P-068: 商品データ（df_product）の各商品について、消費税率10％の税込み金額を求めよ。1円未満の端数は切り捨てとし、結果を10件表示せよ。ただし、単価（unit_price）には欠損が生じていることに注意せよ。

In [65]:
df_product.clone()
    .select([
        col("product_cd"),
        col("unit_price"),
        (col("unit_price") * lit(1.1)).floor().alias("tax_price"),
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────┬────────────┬───────────┐
│ product_cd ┆ unit_price ┆ tax_price │
│ ---        ┆ ---        ┆ ---       │
│ str        ┆ i64        ┆ f64       │
╞════════════╪════════════╪═══════════╡
│ P040101001 ┆ 198        ┆ 217.0     │
│ P040101002 ┆ 218        ┆ 239.0     │
│ P040101003 ┆ 230        ┆ 253.0     │
│ P040101004 ┆ 248        ┆ 272.0     │
│ …          ┆ …          ┆ …         │
│ P040101007 ┆ 338        ┆ 371.0     │
│ P040101008 ┆ 420        ┆ 462.0     │
│ P040101009 ┆ 498        ┆ 547.0     │
│ P040101010 ┆ 580        ┆ 638.0     │
└────────────┴────────────┴───────────┘

---
> P-069: レシート明細データ（df_receipt）と商品データ（df_product）を結合し、顧客毎に全商品の売上金額合計と、カテゴリ大区分コード（category_major_cd）が"07"（瓶詰缶詰）の売上金額合計を計算の上、両者の比率を求めよ。抽出対象はカテゴリ大区分コード"07"（瓶詰缶詰）の売上実績がある顧客のみとし、結果を10件表示せよ。

In [72]:
df_receipt.clone().inner_join(df_product.clone(), col("product_cd"), col("product_cd"))
    .groupby(["customer_id"]).agg([
        col("amount").sum().alias("sum_all"),
        col("amount").filter(col("category_major_cd").eq(lit("07"))).sum().alias("sum_07"),
    ])
    .with_column(
        (col("sum_07") / col("sum_all").cast(DataType::Float64)).alias("sales_rate")
    )
    .filter(col("sum_07").gt(lit(0)))
    .sort(
        "customer_id",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    )
    .collect()
    .unwrap()
    .head(Some(10))


shape: (10, 4)
┌────────────────┬─────────┬────────┬────────────┐
│ customer_id    ┆ sum_all ┆ sum_07 ┆ sales_rate │
│ ---            ┆ ---     ┆ ---    ┆ ---        │
│ str            ┆ i64     ┆ i64    ┆ f64        │
╞════════════════╪═════════╪════════╪════════════╡
│ CS001113000004 ┆ 1298    ┆ 1298   ┆ 1.0        │
│ CS001114000005 ┆ 626     ┆ 486    ┆ 0.776358   │
│ CS001115000010 ┆ 3044    ┆ 2694   ┆ 0.88502    │
│ CS001205000004 ┆ 1988    ┆ 346    ┆ 0.174044   │
│ …              ┆ …       ┆ …      ┆ …          │
│ CS001212000031 ┆ 296     ┆ 296    ┆ 1.0        │
│ CS001212000046 ┆ 228     ┆ 108    ┆ 0.473684   │
│ CS001212000070 ┆ 456     ┆ 308    ┆ 0.675439   │
│ CS001213000018 ┆ 243     ┆ 145    ┆ 0.596708   │
└────────────────┴─────────┴────────┴────────────┘

---
> P-070: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過日数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。

In [102]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%Y%m%d".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .select([col("customer_id"), col("sales_ymd")])
    .unique(
        Some(vec!["customer_id".to_string(), "sales_ymd".to_string()]),
        UniqueKeepStrategy::First,
    )
    .left_join(df_customer.clone(), col("customer_id"), col("customer_id"))
    .select([
        col("customer_id"),
        col("sales_ymd").cast(DataType::Utf8).str().to_date(options.clone()),
        col("application_date").str().to_date(options.clone()),
    ])
    .with_column((col("sales_ymd") - col("application_date")).alias("elapsed_days"))
    .collect()
    .unwrap()
    .head(Some(10))


shape: (10, 4)
┌────────────────┬────────────┬──────────────────┬──────────────┐
│ customer_id    ┆ sales_ymd  ┆ application_date ┆ elapsed_days │
│ ---            ┆ ---        ┆ ---              ┆ ---          │
│ str            ┆ date       ┆ date             ┆ duration[ms] │
╞════════════════╪════════════╪══════════════════╪══════════════╡
│ CS006214000001 ┆ 2018-11-03 ┆ 2015-02-01       ┆ 1371d        │
│ ZZ000000000000 ┆ 2017-05-04 ┆ null             ┆ null         │
│ CS026515000042 ┆ 2019-06-03 ┆ 2015-07-21       ┆ 1413d        │
│ CS026414000062 ┆ 2017-03-06 ┆ 2015-06-10       ┆ 635d         │
│ …              ┆ …          ┆ …                ┆ …            │
│ ZZ000000000000 ┆ 2017-08-29 ┆ null             ┆ null         │
│ ZZ000000000000 ┆ 2019-06-23 ┆ null             ┆ null         │
│ CS018205000001 ┆ 2018-09-11 ┆ 2015-05-02       ┆ 1228d        │
│ CS008515000124 ┆ 2019-05-09 ┆ 2015-09-16       ┆ 1331d        │
└────────────────┴────────────┴──────────────────┴───────────

---
> P-071: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過月数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1ヶ月未満は切り捨てること。

In [8]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%Y%m%d".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .select([col("customer_id"), col("sales_ymd")])
    .unique(
        Some(vec!["customer_id".to_string(), "sales_ymd".to_string()]),
        UniqueKeepStrategy::First,
    )
    .left_join(df_customer.clone(), col("customer_id"), col("customer_id"))
    .select([
        col("customer_id"),
        col("sales_ymd").cast(DataType::Utf8).str().to_date(options.clone()),
        col("application_date").str().to_date(options.clone()),
    ])
    .with_column(((col("sales_ymd") - col("application_date")).cast(DataType::Int64) / (lit(2_629_746_000 as i64))).alias("elapsed_days"))
    .filter(col("customer_id").eq(lit("CS006214000001")))
    .collect()
    .unwrap()
    .head(Some(10))


shape: (10, 4)
┌────────────────┬────────────┬──────────────────┬──────────────┐
│ customer_id    ┆ sales_ymd  ┆ application_date ┆ elapsed_days │
│ ---            ┆ ---        ┆ ---              ┆ ---          │
│ str            ┆ date       ┆ date             ┆ i64          │
╞════════════════╪════════════╪══════════════════╪══════════════╡
│ CS006214000001 ┆ 2018-11-03 ┆ 2015-02-01       ┆ 45           │
│ CS006214000001 ┆ 2019-09-08 ┆ 2015-02-01       ┆ 55           │
│ CS006214000001 ┆ 2018-01-31 ┆ 2015-02-01       ┆ 35           │
│ CS006214000001 ┆ 2019-04-10 ┆ 2015-02-01       ┆ 50           │
│ …              ┆ …          ┆ …                ┆ …            │
│ CS006214000001 ┆ 2019-06-01 ┆ 2015-02-01       ┆ 51           │
│ CS006214000001 ┆ 2017-06-08 ┆ 2015-02-01       ┆ 28           │
│ CS006214000001 ┆ 2018-10-28 ┆ 2015-02-01       ┆ 44           │
│ CS006214000001 ┆ 2017-07-05 ┆ 2015-02-01       ┆ 29           │
└────────────────┴────────────┴──────────────────┴───────────

---
> P-072: レシート明細データ（df_receipt）の売上日（df_customer）に対し、顧客データ（df_customer）の会員申込日（application_date）からの経過年数を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。1年未満は切り捨てること。

In [9]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%Y%m%d".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .select([col("customer_id"), col("sales_ymd")])
    .unique(
        Some(vec!["customer_id".to_string(), "sales_ymd".to_string()]),
        UniqueKeepStrategy::First,
    )
    .left_join(df_customer.clone(), col("customer_id"), col("customer_id"))
    .select([
        col("customer_id"),
        col("sales_ymd").cast(DataType::Utf8).str().to_date(options.clone()),
        col("application_date").str().to_date(options.clone()),
    ])
    .with_column(((col("sales_ymd") - col("application_date")).cast(DataType::Int64) / (lit(31_536_000_000 as i64))).alias("elapsed_years"))
    .filter(col("customer_id").eq(lit("CS006214000001")))
    .collect()
    .unwrap()
    .head(Some(10))


shape: (10, 4)
┌────────────────┬────────────┬──────────────────┬──────────────┐
│ customer_id    ┆ sales_ymd  ┆ application_date ┆ elapsed_days │
│ ---            ┆ ---        ┆ ---              ┆ ---          │
│ str            ┆ date       ┆ date             ┆ i64          │
╞════════════════╪════════════╪══════════════════╪══════════════╡
│ CS006214000001 ┆ 2018-11-03 ┆ 2015-02-01       ┆ 3            │
│ CS006214000001 ┆ 2017-05-09 ┆ 2015-02-01       ┆ 2            │
│ CS006214000001 ┆ 2018-01-31 ┆ 2015-02-01       ┆ 3            │
│ CS006214000001 ┆ 2017-07-05 ┆ 2015-02-01       ┆ 2            │
│ …              ┆ …          ┆ …                ┆ …            │
│ CS006214000001 ┆ 2019-04-10 ┆ 2015-02-01       ┆ 4            │
│ CS006214000001 ┆ 2019-05-07 ┆ 2015-02-01       ┆ 4            │
│ CS006214000001 ┆ 2017-06-08 ┆ 2015-02-01       ┆ 2            │
│ CS006214000001 ┆ 2018-10-28 ┆ 2015-02-01       ┆ 3            │
└────────────────┴────────────┴──────────────────┴───────────

---
> P-073: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、顧客データ（df_customer）の会員申込日（application_date）からのエポック秒による経過時間を計算し、顧客ID（customer_id）、売上日、会員申込日とともに10件表示せよ（なお、sales_ymdは数値、application_dateは文字列でデータを保持している点に注意）。なお、時間情報は保有していないため各日付は0時0分0秒を表すものとする。

In [22]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%Y%m%d".into()),
    strict: false,
    exact: true,
    cache: true,
};

df_receipt.clone()
    .select([col("customer_id"), col("sales_ymd")])
    .unique(
        Some(vec!["customer_id".to_string(), "sales_ymd".to_string()]),
        UniqueKeepStrategy::First,
    )
    .left_join(df_customer.clone(), col("customer_id"), col("customer_id"))
    .select([
        col("customer_id"),
        col("sales_ymd").cast(DataType::Utf8).str().to_date(options.clone()),
        col("application_date").str().to_date(options.clone()),
    ])
    .with_column(((col("sales_ymd").dt().timestamp(TimeUnit::Milliseconds) - col("application_date").dt().timestamp(TimeUnit::Milliseconds)) / lit(1000)).alias("elapsed_epoch"))
    .filter(col("customer_id").eq(lit("CS006214000001")))
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 4)
┌────────────────┬────────────┬──────────────────┬───────────────┐
│ customer_id    ┆ sales_ymd  ┆ application_date ┆ elapsed_epoch │
│ ---            ┆ ---        ┆ ---              ┆ ---           │
│ str            ┆ date       ┆ date             ┆ i64           │
╞════════════════╪════════════╪══════════════════╪═══════════════╡
│ CS006214000001 ┆ 2019-09-08 ┆ 2015-02-01       ┆ 145152000     │
│ CS006214000001 ┆ 2017-07-05 ┆ 2015-02-01       ┆ 76464000      │
│ CS006214000001 ┆ 2019-05-07 ┆ 2015-02-01       ┆ 134438400     │
│ CS006214000001 ┆ 2017-05-09 ┆ 2015-02-01       ┆ 71539200      │
│ …              ┆ …          ┆ …                ┆ …             │
│ CS006214000001 ┆ 2019-04-10 ┆ 2015-02-01       ┆ 132105600     │
│ CS006214000001 ┆ 2018-11-03 ┆ 2015-02-01       ┆ 118454400     │
│ CS006214000001 ┆ 2017-06-08 ┆ 2015-02-01       ┆ 74131200      │
│ CS006214000001 ┆ 2018-01-31 ┆ 2015-02-01       ┆ 94608000      │
└────────────────┴────────────┴────────────────

---
> P-074: レシート明細データ（df_receipt）の売上日（sales_ymd）に対し、当該週の月曜日からの経過日数を計算し、売上日、直前の月曜日付とともに10件表示せよ（sales_ymdは数値でデータを保持している点に注意）。

In [11]:
let options: StrptimeOptions = StrptimeOptions {
    format: Some("%Y%m%d".into()),
    strict: false,
    exact: true,
    cache: true,
};

// let truncate_options = TruncateOptions {
//     every: "w",
//     offset: "",
//     use_earliest: true,
// }

df_receipt.clone()
    .select([
        col("sales_ymd").cast(DataType::Utf8).str().to_date(options.clone()),
    ])
    .with_columns([
        (col("sales_ymd").dt().weekday() - lit(1)).alias("elapsed_days"),
        col("sales_ymd").dt().truncate("1w", "0").alias("monday"), 
    ])
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 3)
┌────────────┬──────────────┬────────────┐
│ sales_ymd  ┆ elapsed_days ┆ monday     │
│ ---        ┆ ---          ┆ ---        │
│ date       ┆ u32          ┆ date       │
╞════════════╪══════════════╪════════════╡
│ 2018-11-03 ┆ 5            ┆ 2018-10-29 │
│ 2018-11-18 ┆ 6            ┆ 2018-11-12 │
│ 2017-07-12 ┆ 2            ┆ 2017-07-10 │
│ 2019-02-05 ┆ 1            ┆ 2019-02-04 │
│ …          ┆ …            ┆ …          │
│ 2018-12-05 ┆ 2            ┆ 2018-12-03 │
│ 2019-09-22 ┆ 6            ┆ 2019-09-16 │
│ 2017-05-04 ┆ 3            ┆ 2017-05-01 │
│ 2019-10-10 ┆ 3            ┆ 2019-10-07 │
└────────────┴──────────────┴────────────┘

---
> P-075: 顧客データ（df_customer）からランダムに1%のデータを抽出し、先頭から10件表示せよ。

In [33]:
df_customer.clone()
    .collect()
    .unwrap()
    .sample_frac(
        0.01,
        true,
        false,
        None,
    )?.head(Some(10))

shape: (10, 11)
┌────────────┬────────────┬───────────┬────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ customer_n ┆ gender_cd ┆ gender ┆ … ┆ address   ┆ applicati ┆ applicati ┆ status_cd │
│ d          ┆ ame        ┆ ---       ┆ ---    ┆   ┆ ---       ┆ on_store_ ┆ on_date   ┆ ---       │
│ ---        ┆ ---        ┆ str       ┆ str    ┆   ┆ str       ┆ cd        ┆ ---       ┆ str       │
│ str        ┆ str        ┆           ┆        ┆   ┆           ┆ ---       ┆ str       ┆           │
│            ┆            ┆           ┆        ┆   ┆           ┆ str       ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ CS00350300 ┆ 阿部 正義  ┆ 0         ┆ 男性   ┆ … ┆ 東京都狛  ┆ S13003    ┆ 20160818  ┆ 0-0000000 │
│ 0057       ┆            ┆           ┆        ┆   ┆ 江市和泉  ┆           ┆           ┆ 0-0       │
│            ┆            ┆           ┆        ┆   ┆ 本町***** ┆           ┆           ┆   

---
> P-076: 顧客データ（df_customer）から性別コード（gender_cd）の割合に基づきランダムに10%のデータを層化抽出し、性別コードごとに件数を集計せよ。

In [40]:
fn sample_frac_gb(x: DataFrame) -> DataFrame {
    x.sample_frac(0.1, true, false, None).unwrap()
}

df_customer.clone()
    .groupby(["gender_cd"]).apply(|x| Ok(sample_frac_gb(x)), df_customer.schema().unwrap())
    .groupby(["gender_cd"]).agg([col("customer_id").count()])
    // .select([
    //     col("gender_cd").value_counts(true, true),
    // ]) こちらでも良いが出力形式がPythonと違う
    .collect()
    .unwrap()

shape: (3, 2)
┌───────────┬─────────────┐
│ gender_cd ┆ customer_id │
│ ---       ┆ ---         │
│ str       ┆ u32         │
╞═══════════╪═════════════╡
│ 1         ┆ 1791        │
│ 0         ┆ 298         │
│ 9         ┆ 107         │
└───────────┴─────────────┘

---
> P-077: レシート明細データ（df_receipt）の売上金額を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。なお、外れ値は売上金額合計を対数化したうえで平均と標準偏差を計算し、その平均から3σを超えて離れたものとする（自然対数と常用対数のどちらでも可）。結果は10件表示せよ。

In [50]:
df_receipt.clone()
    .groupby(["customer_id"]).agg([col("amount").sum()])
    .with_column((col("amount") - lit(0.5)).log1p().alias("log_sum_amount"))
    .with_column(
        ((col("log_sum_amount") - col("log_sum_amount").mean()).abs() / col("log_sum_amount").std(0))
        .alias("log_sum_amount_ss")
    )
    .filter(
        (col("log_sum_amount") - col("log_sum_amount").mean()).abs()
        .gt(
            col("log_sum_amount").std(0) * lit(3.0)
        )
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (1, 4)
┌────────────────┬──────────┬────────────────┬───────────────────┐
│ customer_id    ┆ amount   ┆ log_sum_amount ┆ log_sum_amount_ss │
│ ---            ┆ ---      ┆ ---            ┆ ---               │
│ str            ┆ i64      ┆ f64            ┆ f64               │
╞════════════════╪══════════╪════════════════╪═══════════════════╡
│ ZZ000000000000 ┆ 12395003 ┆ 16.332804      ┆ 7.967822          │
└────────────────┴──────────┴────────────────┴───────────────────┘

---
> P-078: レシート明細データ（df_receipt）の売上金額（amount）を顧客単位に合計し、合計した売上金額の外れ値を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、ここでは外れ値を第1四分位と第3四分位の差であるIQRを用いて、「第1四分位数-1.5×IQR」を下回るもの、または「第3四分位数+1.5×IQR」を超えるものとする。結果は10件表示せよ。

In [64]:
df_receipt.clone()
    .filter(col("customer_id").str().starts_with(lit("Z")).not())
    .groupby(["customer_id"]).agg([col("amount").sum()])
    .with_columns([
        col("amount").quantile(lit(0.25), QuantileInterpolOptions::Nearest).alias("1qtr"),
        col("amount").quantile(lit(0.75), QuantileInterpolOptions::Nearest).alias("3qtr"),
    ])
    .with_column((col("3qtr") - col("1qtr")).alias("iqr"))
    .filter(
        col("amount").lt(col("1qtr") - lit(1.5) * col("iqr")).or(col("amount").gt(col("3qtr") + lit(1.5) * col("iqr")))
    )
    .sort(
        "customer_id",
        SortOptions{
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    )
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 5)
┌────────────────┬────────┬───────┬────────┬────────┐
│ customer_id    ┆ amount ┆ 1qtr  ┆ 3qtr   ┆ iqr    │
│ ---            ┆ ---    ┆ ---   ┆ ---    ┆ ---    │
│ str            ┆ i64    ┆ f64   ┆ f64    ┆ f64    │
╞════════════════╪════════╪═══════╪════════╪════════╡
│ CS001414000048 ┆ 8584   ┆ 548.0 ┆ 3650.0 ┆ 3102.0 │
│ CS001605000009 ┆ 18925  ┆ 548.0 ┆ 3650.0 ┆ 3102.0 │
│ CS002415000594 ┆ 9568   ┆ 548.0 ┆ 3650.0 ┆ 3102.0 │
│ CS004414000181 ┆ 9584   ┆ 548.0 ┆ 3650.0 ┆ 3102.0 │
│ …              ┆ …      ┆ …     ┆ …      ┆ …      │
│ CS006414000029 ┆ 9179   ┆ 548.0 ┆ 3650.0 ┆ 3102.0 │
│ CS006415000105 ┆ 10042  ┆ 548.0 ┆ 3650.0 ┆ 3102.0 │
│ CS006415000147 ┆ 12723  ┆ 548.0 ┆ 3650.0 ┆ 3102.0 │
│ CS006415000157 ┆ 10648  ┆ 548.0 ┆ 3650.0 ┆ 3102.0 │
└────────────────┴────────┴───────┴────────┴────────┘

---
> P-079: 商品データ（df_product）の各項目に対し、欠損数を確認せよ。

In [66]:
df_product.clone()
    .null_count()
    .collect()
    .unwrap()

shape: (1, 6)
┌────────────┬───────────────────┬────────────────────┬───────────────────┬────────────┬───────────┐
│ product_cd ┆ category_major_cd ┆ category_medium_cd ┆ category_small_cd ┆ unit_price ┆ unit_cost │
│ ---        ┆ ---               ┆ ---                ┆ ---               ┆ ---        ┆ ---       │
│ u32        ┆ u32               ┆ u32                ┆ u32               ┆ u32        ┆ u32       │
╞════════════╪═══════════════════╪════════════════════╪═══════════════════╪════════════╪═══════════╡
│ 0          ┆ 0                 ┆ 0                  ┆ 0                 ┆ 7          ┆ 7         │
└────────────┴───────────────────┴────────────────────┴───────────────────┴────────────┴───────────┘

---
> P-080: 商品データ（df_product）のいずれかの項目に欠損が発生しているレコードを全て削除した新たな商品データを作成せよ。なお、削除前後の件数を表示させ、079で確認した件数だけ減少していることも確認すること。

In [79]:
println!("削除前: {:?}", df_product.clone().collect().unwrap().height());

println!("削除後: {:?}", df_product.clone().drop_nulls(Some(vec![col("*")])).collect().unwrap().height());

削除前: 10030
削除後: 10023


---
> P-081: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの平均値で補完した新たな商品データを作成せよ。なお、平均値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [22]:
let tmp_df = df_product.clone()
    .collect()
    .unwrap()
    .fill_null(FillNullStrategy::Mean)?;

println!("{:?}", &tmp_df);
tmp_df.null_count()

// let tmp_df = df_product.clone()
//     .with_columns([
//         col("unit_price").fill_null(col("unit_price").mean().cast(DataType::Int32)),
//         col("unit_cost").fill_null(col("unit_cost").mean().cast(DataType::Int32)),
//     ]);
// println!("{:?}", tmp_df.clone().collect().unwrap());
// tmp_df.null_count().collect().unwrap()


shape: (10_030, 6)
┌────────────┬───────────────────┬────────────────────┬───────────────────┬────────────┬───────────┐
│ product_cd ┆ category_major_cd ┆ category_medium_cd ┆ category_small_cd ┆ unit_price ┆ unit_cost │
│ ---        ┆ ---               ┆ ---                ┆ ---               ┆ ---        ┆ ---       │
│ str        ┆ str               ┆ str                ┆ str               ┆ i64        ┆ i64       │
╞════════════╪═══════════════════╪════════════════════╪═══════════════════╪════════════╪═══════════╡
│ P040101001 ┆ 04                ┆ 0401               ┆ 040101            ┆ 198        ┆ 149       │
│ P040101002 ┆ 04                ┆ 0401               ┆ 040101            ┆ 218        ┆ 164       │
│ P040101003 ┆ 04                ┆ 0401               ┆ 040101            ┆ 230        ┆ 173       │
│ P040101004 ┆ 04                ┆ 0401               ┆ 040101            ┆ 248        ┆ 186       │
│ …          ┆ …                 ┆ …                  ┆ …               

shape: (1, 6)
┌────────────┬───────────────────┬────────────────────┬───────────────────┬────────────┬───────────┐
│ product_cd ┆ category_major_cd ┆ category_medium_cd ┆ category_small_cd ┆ unit_price ┆ unit_cost │
│ ---        ┆ ---               ┆ ---                ┆ ---               ┆ ---        ┆ ---       │
│ u32        ┆ u32               ┆ u32                ┆ u32               ┆ u32        ┆ u32       │
╞════════════╪═══════════════════╪════════════════════╪═══════════════════╪════════════╪═══════════╡
│ 0          ┆ 0                 ┆ 0                  ┆ 0                 ┆ 0          ┆ 0         │
└────────────┴───────────────────┴────────────────────┴───────────────────┴────────────┴───────────┘

---
> P-082: 単価（unit_price）と原価（unit_cost）の欠損値について、それぞれの中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [25]:
let tmp_df = df_product.clone()
    .with_columns([
        col("unit_price").fill_null(col("unit_price").median().cast(DataType::Int32)).alias("filled_unit_price"),
        col("unit_cost").fill_null(col("unit_cost").median().cast(DataType::Int32)).alias("filled_unit_cost"),
    ]);

println!(
    "{:?}",
    tmp_df.clone()
        .filter(col("unit_price").is_null().or(col("unit_cost").is_null()))
        .collect()
        .unwrap()
);
tmp_df.null_count().collect().unwrap()

shape: (7, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ product_cd ┆ category_m ┆ category_m ┆ category_ ┆ unit_pric ┆ unit_cost ┆ filled_un ┆ filled_un │
│ ---        ┆ ajor_cd    ┆ edium_cd   ┆ small_cd  ┆ e         ┆ ---       ┆ it_price  ┆ it_cost   │
│ str        ┆ ---        ┆ ---        ┆ ---       ┆ ---       ┆ i64       ┆ ---       ┆ ---       │
│            ┆ str        ┆ str        ┆ str       ┆ i64       ┆           ┆ i64       ┆ i64       │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ P040802007 ┆ 04         ┆ 0408       ┆ 040802    ┆ null      ┆ null      ┆ 252       ┆ 189       │
│ P050103021 ┆ 05         ┆ 0501       ┆ 050103    ┆ null      ┆ null      ┆ 252       ┆ 189       │
│ P050405009 ┆ 05         ┆ 0504       ┆ 050405    ┆ null      ┆ null      ┆ 252       ┆ 189       │
│ P060802026 ┆ 06         ┆ 0608       ┆ 060802    ┆ null      ┆ null      ┆ 

shape: (1, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ product_cd ┆ category_m ┆ category_m ┆ category_ ┆ unit_pric ┆ unit_cost ┆ filled_un ┆ filled_un │
│ ---        ┆ ajor_cd    ┆ edium_cd   ┆ small_cd  ┆ e         ┆ ---       ┆ it_price  ┆ it_cost   │
│ u32        ┆ ---        ┆ ---        ┆ ---       ┆ ---       ┆ u32       ┆ ---       ┆ ---       │
│            ┆ u32        ┆ u32        ┆ u32       ┆ u32       ┆           ┆ u32       ┆ u32       │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 0          ┆ 0          ┆ 0          ┆ 0         ┆ 7         ┆ 7         ┆ 0         ┆ 0         │
└────────────┴────────────┴────────────┴───────────┴───────────┴───────────┴───────────┴───────────┘

---
> P-083: 単価（unit_price）と原価（unit_cost）の欠損値について、各商品のカテゴリ小区分コード（category_small_cd）ごとに算出した中央値で補完した新たな商品データを作成せよ。なお、中央値については1円未満を丸めること（四捨五入または偶数への丸めで良い）。補完実施後、各項目について欠損が生じていないことも確認すること。

In [26]:
let tmp_df: LazyFrame = df_product.clone()
    .with_columns([
        coalesce(&[
            col("unit_cost"),
            median("unit_cost").over([col("category_small_cd")]).cast(DataType::Int64)
        ]).alias("filled_unit_price"),
        coalesce(&[
            col("unit_price"),
            median("unit_price").over([col("category_small_cd")]).cast(DataType::Int64)
        ]).alias("filled_unit_cost"),
    ]);

println!(
    "{:?}",
    tmp_df.clone()
        .filter(col("unit_price").is_null().or(col("unit_cost").is_null()))
        .collect()
        .unwrap()
);
tmp_df.null_count().collect().unwrap()

shape: (7, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ product_cd ┆ category_m ┆ category_m ┆ category_ ┆ unit_pric ┆ unit_cost ┆ filled_un ┆ filled_un │
│ ---        ┆ ajor_cd    ┆ edium_cd   ┆ small_cd  ┆ e         ┆ ---       ┆ it_price  ┆ it_cost   │
│ str        ┆ ---        ┆ ---        ┆ ---       ┆ ---       ┆ i64       ┆ ---       ┆ ---       │
│            ┆ str        ┆ str        ┆ str       ┆ i64       ┆           ┆ i64       ┆ i64       │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ P040802007 ┆ 04         ┆ 0408       ┆ 040802    ┆ null      ┆ null      ┆ 235       ┆ 313       │
│ P050103021 ┆ 05         ┆ 0501       ┆ 050103    ┆ null      ┆ null      ┆ 99        ┆ 132       │
│ P050405009 ┆ 05         ┆ 0504       ┆ 050405    ┆ null      ┆ null      ┆ 133       ┆ 178       │
│ P060802026 ┆ 06         ┆ 0608       ┆ 060802    ┆ null      ┆ null      ┆ 

shape: (1, 8)
┌────────────┬────────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┐
│ product_cd ┆ category_m ┆ category_m ┆ category_ ┆ unit_pric ┆ unit_cost ┆ filled_un ┆ filled_un │
│ ---        ┆ ajor_cd    ┆ edium_cd   ┆ small_cd  ┆ e         ┆ ---       ┆ it_price  ┆ it_cost   │
│ u32        ┆ ---        ┆ ---        ┆ ---       ┆ ---       ┆ u32       ┆ ---       ┆ ---       │
│            ┆ u32        ┆ u32        ┆ u32       ┆ u32       ┆           ┆ u32       ┆ u32       │
╞════════════╪════════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 0          ┆ 0          ┆ 0          ┆ 0         ┆ 7         ┆ 7         ┆ 0         ┆ 0         │
└────────────┴────────────┴────────────┴───────────┴───────────┴───────────┴───────────┴───────────┘

---
> P-084: 顧客データ（df_customer）の全顧客に対して全期間の売上金額に占める2019年売上金額の割合を計算し、新たなデータを作成せよ。ただし、売上実績がない場合は0として扱うこと。そして計算した割合が0超のものを抽出し、結果を10件表示せよ。また、作成したデータに欠損が存在しないことを確認せよ。

In [57]:
let tmp_df = df_customer.clone()
    .inner_join(
        df_receipt.clone()
            .with_column(col("amount").fill_null(lit(0)))
            .groupby(["customer_id"]).agg([
                col("amount")
                    .filter(
                        col("sales_ymd").lt(lit(20200101))
                        .and(col("sales_ymd").gt(lit(20181231)))
                    )
                    .sum().alias("amount_2019"),
                col("amount").sum().alias("amount_all"),
            ])
            .with_column(
                (col("amount_2019").cast(DataType::Float32) / col("amount_all"))
                    .alias("amount_rate")
            )
            .filter(col("amount_rate").gt(lit(0))),
        col("customer_id"),
        col("customer_id"),
    )
    .select([
        col("customer_id"),
        col("amount_2019"),
        col("amount_all"),
        col("amount_rate"),
    ]);

println!("{:?}", tmp_df.clone().collect().unwrap().head(Some(10)));
tmp_df.null_count().collect().unwrap()


shape: (10, 4)
┌────────────────┬─────────────┬────────────┬─────────────┐
│ customer_id    ┆ amount_2019 ┆ amount_all ┆ amount_rate │
│ ---            ┆ ---         ┆ ---        ┆ ---         │
│ str            ┆ i64         ┆ i64        ┆ f64         │
╞════════════════╪═════════════╪════════════╪═════════════╡
│ CS031415000172 ┆ 2971        ┆ 5088       ┆ 0.583923    │
│ CS015414000103 ┆ 874         ┆ 3122       ┆ 0.279949    │
│ CS011215000048 ┆ 248         ┆ 3444       ┆ 0.072009    │
│ CS029415000023 ┆ 3767        ┆ 5167       ┆ 0.72905     │
│ …              ┆ …           ┆ …          ┆ …           │
│ CS035513000134 ┆ 463         ┆ 1565       ┆ 0.295847    │
│ CS001515000263 ┆ 216         ┆ 216        ┆ 1.0         │
│ CS006415000279 ┆ 229         ┆ 229        ┆ 1.0         │
│ CS031415000106 ┆ 215         ┆ 7741       ┆ 0.027774    │
└────────────────┴─────────────┴────────────┴─────────────┘


shape: (1, 4)
┌─────────────┬─────────────┬────────────┬─────────────┐
│ customer_id ┆ amount_2019 ┆ amount_all ┆ amount_rate │
│ ---         ┆ ---         ┆ ---        ┆ ---         │
│ u32         ┆ u32         ┆ u32        ┆ u32         │
╞═════════════╪═════════════╪════════════╪═════════════╡
│ 0           ┆ 0           ┆ 0          ┆ 0           │
└─────────────┴─────────────┴────────────┴─────────────┘

---
> P-085: 顧客データ（df_customer）の全顧客に対し、郵便番号（postal_cd）を用いてジオコードデータ（df_geocode）を紐付け、新たな顧客データを作成せよ。ただし、1つの郵便番号（postal_cd）に複数の経度（longitude）、緯度（latitude）情報が紐づく場合は、経度（longitude）、緯度（latitude）の平均値を算出して使用すること。また、作成結果を確認するために結果を10件表示せよ。

In [75]:
let df_customer1 = df_customer.clone()
    .inner_join(
        df_geocode.clone()
            .groupby(["postal_cd"]).agg([
                col("longitude").mean(),
                col("latitude").mean(),
            ]),
        col("postal_cd"),
        col("postal_cd"),
    );

df_customer1.clone()
.filter(
    col("postal_cd").eq(lit("136-0076"))
    .or(col("postal_cd").eq(lit("259-1113")))
) // pythonの解答と同じ情報を出力するための記述
.collect().unwrap().head(Some(10))

shape: (10, 13)
┌────────────┬────────────┬───────────┬────────┬───┬────────────┬───────────┬───────────┬──────────┐
│ customer_i ┆ customer_n ┆ gender_cd ┆ gender ┆ … ┆ applicatio ┆ status_cd ┆ longitude ┆ latitude │
│ d          ┆ ame        ┆ ---       ┆ ---    ┆   ┆ n_date     ┆ ---       ┆ ---       ┆ ---      │
│ ---        ┆ ---        ┆ str       ┆ str    ┆   ┆ ---        ┆ str       ┆ f64       ┆ f64      │
│ str        ┆ str        ┆           ┆        ┆   ┆ str        ┆           ┆           ┆          │
╞════════════╪════════════╪═══════════╪════════╪═══╪════════════╪═══════════╪═══════════╪══════════╡
│ CS02131300 ┆ 大野       ┆ 1         ┆ 女性   ┆ … ┆ 20150905   ┆ 0-0000000 ┆ 139.31779 ┆ 35.41358 │
│ 0114       ┆ あや子     ┆           ┆        ┆   ┆            ┆ 0-0       ┆           ┆          │
│ CS02130300 ┆ 堀 一徳    ┆ 0         ┆ 男性   ┆ … ┆ 20160411   ┆ 0-0000000 ┆ 139.31779 ┆ 35.41358 │
│ 0023       ┆            ┆           ┆        ┆   ┆            ┆ 0-0       ┆          

---
> P-086: 085で作成した緯度経度つき顧客データに対し、会員申込店舗コード（application_store_cd）をキーに店舗データ（df_store）と結合せよ。そして申込み店舗の緯度（latitude）・経度情報（longitude)と顧客住所（address）の緯度・経度を用いて申込み店舗と顧客住所の距離（単位：km）を求め、顧客ID（customer_id）、顧客住所（address）、店舗住所（address）とともに表示せよ。計算式は以下の簡易式で良いものとするが、その他精度の高い方式を利用したライブラリを利用してもかまわない。結果は10件表示せよ。

$$
\text{緯度（ラジアン）}：\phi \\
\text{経度（ラジアン）}：\lambda \\
\text{距離}L = 6371 * \arccos(\sin \phi_1 * \sin \phi_2
+ \cos \phi_1 * \cos \phi_2 * \cos(\lambda_1 − \lambda_2))
$$

In [89]:
use std::f64::consts::PI;

fn distance_expr(lon1_rad: Expr, lat1_rad: Expr, lon2_rad: Expr, lat2_rad: Expr) -> Expr {
    lit(6371) * (
        lat1_rad.clone().sin() * lat2_rad.clone().sin() + 
        lat1_rad.clone().cos() * lat2_rad.clone().cos() * (lon1_rad.clone() - lon2_rad.clone()).cos()
    ).arccos()
}

df_customer1.clone()
    .inner_join(
        df_store.clone(),
        col("application_store_cd"),
        col("store_cd"),
    )
    .select([
        col("customer_id"),
        col("address").alias("customer_address"),
        col("address_right").alias("store_address"),
        distance_expr(
            col("longitude").clone().radians(),
            col("latitude").clone().radians(),
            col("longitude_right").clone().radians(),
            col("latitude_right").clone().radians()
        ).alias("distance")
    ])
    .filter(
        col("customer_address").eq(lit("神奈川県伊勢原市粟窪**********"))
        .or(col("customer_address").eq(lit("神奈川県伊勢原市伊勢原**********")))
    ) // pythonの解答の出力に合わせるために必要
    .collect()
    .unwrap()
    .head(Some(10))

shape: (10, 4)
┌────────────────┬──────────────────────────────────┬──────────────────────────────┬──────────┐
│ customer_id    ┆ customer_address                 ┆ store_address                ┆ distance │
│ ---            ┆ ---                              ┆ ---                          ┆ ---      │
│ str            ┆ str                              ┆ str                          ┆ f64      │
╞════════════════╪══════════════════════════════════╪══════════════════════════════╪══════════╡
│ CS021313000114 ┆ 神奈川県伊勢原市粟窪**********   ┆ 神奈川県伊勢原市伊勢原四丁目 ┆ 1.394409 │
│ CS021303000023 ┆ 神奈川県伊勢原市粟窪**********   ┆ 神奈川県伊勢原市伊勢原四丁目 ┆ 1.394409 │
│ CS021303000007 ┆ 神奈川県伊勢原市粟窪**********   ┆ 神奈川県伊勢原市伊勢原四丁目 ┆ 1.394409 │
│ CS021313000183 ┆ 神奈川県伊勢原市粟窪**********   ┆ 神奈川県伊勢原市伊勢原四丁目 ┆ 1.394409 │
│ …              ┆ …                                ┆ …                            ┆ …        │
│ CS021413000049 ┆ 神奈川県伊勢原市粟窪**********   ┆ 神奈川県伊勢原市伊勢原四丁目 ┆ 1.394409 │
│ CS021313000025 ┆ 神奈川県伊勢原市伊勢原********** ┆ 神奈川県伊勢

---
> P-087: 顧客データ（df_customer）では、異なる店舗での申込みなどにより同一顧客が複数登録されている。名前（customer_name）と郵便番号（postal_cd）が同じ顧客は同一顧客とみなして1顧客1レコードとなるように名寄せした名寄顧客データを作成し、顧客データの件数、名寄顧客データの件数、重複数を算出せよ。ただし、同一顧客に対しては売上金額合計が最も高いものを残し、売上金額合計が同一もしくは売上実績がない顧客については顧客ID（customer_id）の番号が小さいものを残すこととする。

In [113]:
let df_customer_u = df_customer.clone().left_join(
        df_receipt.clone().groupby(["customer_id"]).agg([col("amount").sum()]),
        col("customer_id"),
        col("customer_id"),
    ).sort_by_exprs(
        vec![col("amount"), col("customer_id")],
        vec![true, false],
        true,
        true,
    ).unique(
        Some(vec!["customer_name".to_string(), "postal_cd".to_string()]),
        UniqueKeepStrategy::First,
    );

println!("{:?}", df_customer_u.clone().collect().unwrap());

let data_count_c = df_customer.clone().collect().unwrap().height();
let data_count_u = df_customer_u.clone().collect().unwrap().height();
println!(
    "df_customer_cnt:{}\ndf_customer_unique_cnt: {}\ndiff: {}",
    &data_count_c,
    &data_count_u,
    &data_count_c - &data_count_u
);

shape: (21_941, 12)
┌────────────┬────────────┬───────────┬────────┬───┬────────────┬────────────┬────────────┬────────┐
│ customer_i ┆ customer_n ┆ gender_cd ┆ gender ┆ … ┆ applicatio ┆ applicatio ┆ status_cd  ┆ amount │
│ d          ┆ ame        ┆ ---       ┆ ---    ┆   ┆ n_store_cd ┆ n_date     ┆ ---        ┆ ---    │
│ ---        ┆ ---        ┆ str       ┆ str    ┆   ┆ ---        ┆ ---        ┆ str        ┆ i64    │
│ str        ┆ str        ┆           ┆        ┆   ┆ str        ┆ str        ┆            ┆        │
╞════════════╪════════════╪═══════════╪════════╪═══╪════════════╪════════════╪════════════╪════════╡
│ CS01741500 ┆ 福士 千夏  ┆ 1         ┆ 女性   ┆ … ┆ S13017     ┆ 20151209   ┆ F-20101006 ┆ 23086  │
│ 0097       ┆            ┆           ┆        ┆   ┆            ┆            ┆ -F         ┆        │
│ CS01541500 ┆ 岩淵       ┆ 1         ┆ 女性   ┆ … ┆ S13015     ┆ 20150322   ┆ F-20101014 ┆ 20153  │
│ 0185       ┆ はるみ     ┆           ┆        ┆   ┆            ┆            ┆ -F   

---
> P-088: 087で作成したデータを元に、顧客データに統合名寄IDを付与したデータを作成せよ。ただし、統合名寄IDは以下の仕様で付与するものとする。
>
> - 重複していない顧客：顧客ID（customer_id）を設定
> - 重複している顧客：前設問で抽出したレコードの顧客IDを設定
> 
> 顧客IDのユニーク件数と、統合名寄IDのユニーク件数の差も確認すること。

In [122]:
let df_customer_n = df_customer.clone()
    .join(
        df_customer_u.clone()
            .select([
                col("customer_name"),
                col("postal_cd"),
                col("customer_id").alias("integration_id"),
            ]),
        [col("customer_name"), col("postal_cd")],
        [col("customer_name"), col("postal_cd")],
        JoinArgs::new(JoinType::Inner),
    );

println!("{:?}", df_customer_n.clone().collect().unwrap());

df_customer_n
    .select([
        col("customer_id").n_unique(),
        col("integration_id").n_unique(),
        (col("customer_id").n_unique() - col("integration_id").n_unique()).alias("diff")
    ])
    .collect()
    .unwrap()

shape: (21_971, 12)
┌────────────┬────────────┬───────────┬────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ customer_i ┆ customer_n ┆ gender_cd ┆ gender ┆ … ┆ applicati ┆ applicati ┆ status_cd ┆ integrati │
│ d          ┆ ame        ┆ ---       ┆ ---    ┆   ┆ on_store_ ┆ on_date   ┆ ---       ┆ on_id     │
│ ---        ┆ ---        ┆ str       ┆ str    ┆   ┆ cd        ┆ ---       ┆ str       ┆ ---       │
│ str        ┆ str        ┆           ┆        ┆   ┆ ---       ┆ str       ┆           ┆ str       │
│            ┆            ┆           ┆        ┆   ┆ str       ┆           ┆           ┆           │
╞════════════╪════════════╪═══════════╪════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ CS02131300 ┆ 大野       ┆ 1         ┆ 女性   ┆ … ┆ S14021    ┆ 20150905  ┆ 0-0000000 ┆ CS0213130 │
│ 0114       ┆ あや子     ┆           ┆        ┆   ┆           ┆           ┆ 0-0       ┆ 00114     │
│ CS03761300 ┆ 六角 雅彦  ┆ 9         ┆ 不明   ┆ … ┆ S13037    ┆ 20150414  ┆ 0-00000

shape: (1, 3)
┌─────────────┬────────────────┬──────┐
│ customer_id ┆ integration_id ┆ diff │
│ ---         ┆ ---            ┆ ---  │
│ u32         ┆ u32            ┆ u32  │
╞═════════════╪════════════════╪══════╡
│ 21971       ┆ 21941          ┆ 30   │
└─────────────┴────────────────┴──────┘

---
> P-089: 売上実績がある顧客を、予測モデル構築のため学習用データとテスト用データに分割したい。それぞれ8:2の割合でランダムにデータを分割せよ。

In [132]:
fn convert_features_to_matrix(df: &DataFrame) -> Result<DenseMatrix<f64>> {
    let nrows = df.height();
    let ncols = df.width();

    let features_res = df.to_ndarray::<Float64Type>().unwrap();
    let mut xmatrix: DenseMatrix<f64> = BaseMatrix::zeros(nrows, ncols);

    let mut col: u32 = 0;
    let mut row: u32 = 0;

    for val in features_res.iter() {
        let m_row = usize::try_from(row).unwrap();
        let m_col = usize::try_from(col).unwrap();
        xmatrix.set(m_row, m_col, *val);
        if m_col == ncols - 1 {
            row += 1;
            col = 0;
        } else {
            col += 1;
        }
    }
    Ok(xmatrix)
}

let df_sales_customer = df_receipt.clone()
    .groupby(["customer_id"]).agg([col("amount").sum()])
    .filter(col("amount") > 0);

let df_train =

let df_test = df_sales_customer.clone()
    .with_row_count("id")
    .with_column(col("id").shuffle() < 0.8 * all().len())
    .partition_by("id");

println(
    "{:?}\n{:?}\n{:?}"
    df_train.shape(),
    df_test.shape(),
    df_sales_customer.shape(),
)

Error: Compilation failed, but no parsable errors were found. STDERR:
   Compiling num-traits v0.2.16
   Compiling num-integer v0.1.45
   Compiling num-bigint v0.4.3
   Compiling num-iter v0.1.43
   Compiling num-rational v0.4.1
   Compiling chrono v0.4.26
   Compiling atoi v2.0.0
   Compiling polars-utils v0.31.1
   Compiling rand_distr v0.4.3
   Compiling argminmax v0.6.1
   Compiling num-complex v0.4.3
   Compiling arrow2 v0.17.3
   Compiling now v0.1.3
   Compiling approx v0.5.1
   Compiling num v0.4.1
   Compiling smartcore v0.3.2
   Compiling polars-error v0.31.1
   Compiling polars-arrow v0.31.1
   Compiling polars-row v0.31.1
   Compiling polars-core v0.31.1
   Compiling polars-ops v0.31.1
   Compiling polars-time v0.31.1
   Compiling polars-io v0.31.1
   Compiling polars-plan v0.31.1
   Compiling polars-pipe v0.31.1
   Compiling polars-lazy v0.31.1
   Compiling polars-sql v0.31.1
   Compiling polars v0.31.1
error: could not compile `polars-core` (lib)

Caused by:
  process did

---
> P-090: レシート明細データ（df_receipt）は2017年1月1日〜2019年10月31日までのデータを有している。売上金額（amount）を月次で集計し、学習用に12ヶ月、テスト用に6ヶ月の時系列モデル構築用データを3セット作成せよ。

In [127]:
let df_ts_amount = df_receipt.clone()
    .groupby([col("sales_ymd") / lit(100)]).agg([col("amount").sum()])
    .sort(
        "sales_ymd",
        SortOptions {
            descending: false,
            nulls_last: true,
            multithreaded: true,
            maintain_order: true
        }
    );

fn split_data(df: LazyFrame, train_size: u32, test_size: u32, slide_window: i64, start_point: i64) -> (LazyFrame, LazyFrame) {
    let train_start = start_point * slide_window;
    let test_start = train_start + train_size as i64;
    (df.clone().slice(train_start, train_size), df.clone().slice(test_start, test_size))
}

let (df_train_1, df_test_1) = split_data(df_ts_amount.clone(), 12, 6, 6, 0);
let (df_train_2, df_test_2) = split_data(df_ts_amount.clone(), 12, 6, 6, 1);
let (df_train_3, df_test_3) = split_data(df_ts_amount.clone(), 12, 6, 6, 2);

println!(
    "df_train1:\n{:?}\ndf_test1:\n{:?}\ndf_train2:\n{:?}\ndf_test2:\n{:?}\ndf_train3:\n{:?}\ndf_test3:\n{:?}\n",    
    df_train_1.collect().unwrap(),
    df_test_1.collect().unwrap(),
    df_train_2.collect().unwrap(),
    df_test_2.collect().unwrap(),
    df_train_3.collect().unwrap(),
    df_test_3.collect().unwrap(),
);

df_train1:
shape: (12, 2)
┌───────────┬────────┐
│ sales_ymd ┆ amount │
│ ---       ┆ ---    │
│ i64       ┆ i64    │
╞═══════════╪════════╡
│ 201701    ┆ 902056 │
│ 201702    ┆ 764413 │
│ 201703    ┆ 962945 │
│ 201704    ┆ 847566 │
│ …         ┆ …      │
│ 201709    ┆ 902037 │
│ 201710    ┆ 905739 │
│ 201711    ┆ 932157 │
│ 201712    ┆ 939654 │
└───────────┴────────┘
df_test1:
shape: (6, 2)
┌───────────┬─────────┐
│ sales_ymd ┆ amount  │
│ ---       ┆ ---     │
│ i64       ┆ i64     │
╞═══════════╪═════════╡
│ 201801    ┆ 944509  │
│ 201802    ┆ 864128  │
│ 201803    ┆ 946588  │
│ 201804    ┆ 937099  │
│ 201805    ┆ 1004438 │
│ 201806    ┆ 1012329 │
└───────────┴─────────┘
df_train2:
shape: (12, 2)
┌───────────┬─────────┐
│ sales_ymd ┆ amount  │
│ ---       ┆ ---     │
│ i64       ┆ i64     │
╞═══════════╪═════════╡
│ 201707    ┆ 959205  │
│ 201708    ┆ 954836  │
│ 201709    ┆ 902037  │
│ 201710    ┆ 905739  │
│ …         ┆ …       │
│ 201803    ┆ 946588  │
│ 201804    ┆ 937099  │
│ 2

---
> P-091: 顧客データ（df_customer）の各顧客に対し、売上実績がある顧客数と売上実績がない顧客数が1:1となるようにアンダーサンプリングで抽出せよ。

---
> P-092: 顧客データ（df_customer）の性別について、第三正規形へと正規化せよ。

In [145]:
let df_gender_std: LazyFrame = df_customer.clone()
    .select([col("gender_cd"), col("gender")])
    .unique(
        Some(vec!["gender_cd".to_string(), "gender".to_string()]),
        UniqueKeepStrategy::First
    );
let df_customer_std: LazyFrame = df_customer.clone().drop_columns(["gender"]);

In [146]:
df_customer_std.collect().unwrap().head(Some(3))

shape: (3, 10)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ customer_ ┆ customer_ ┆ gender_cd ┆ birth_day ┆ … ┆ address   ┆ applicati ┆ applicati ┆ status_c │
│ id        ┆ name      ┆ ---       ┆ ---       ┆   ┆ ---       ┆ on_store_ ┆ on_date   ┆ d        │
│ ---       ┆ ---       ┆ str       ┆ date      ┆   ┆ str       ┆ cd        ┆ ---       ┆ ---      │
│ str       ┆ str       ┆           ┆           ┆   ┆           ┆ ---       ┆ str       ┆ str      │
│           ┆           ┆           ┆           ┆   ┆           ┆ str       ┆           ┆          │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ CS0213130 ┆ 大野      ┆ 1         ┆ 1981-04-2 ┆ … ┆ 神奈川県  ┆ S14021    ┆ 20150905  ┆ 0-000000 │
│ 00114     ┆ あや子    ┆           ┆ 9         ┆   ┆ 伊勢原市  ┆           ┆           ┆ 00-0     │
│           ┆           ┆           ┆           ┆   ┆ 粟窪***** ┆           ┆           ┆  

In [147]:
df_gender_std.collect().unwrap().head(Some(3))

shape: (3, 2)
┌───────────┬────────┐
│ gender_cd ┆ gender │
│ ---       ┆ ---    │
│ str       ┆ str    │
╞═══════════╪════════╡
│ 1         ┆ 女性   │
│ 9         ┆ 不明   │
│ 0         ┆ 男性   │
└───────────┴────────┘

---
> P-093: 商品データ（df_product）では各カテゴリのコード値だけを保有し、カテゴリ名は保有していない。カテゴリデータ（df_category）と組み合わせて非正規化し、カテゴリ名を保有した新たな商品データを作成せよ。

In [167]:
let df_product_full = df_product.clone().left_join(
        df_category.clone().select([col("category_small_cd"), col("^category_.*_name$")]), 
        col("category_small_cd"),
        col("category_small_cd"),
    );

df_product_full.clone().collect().unwrap()

shape: (10_030, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ product_c ┆ category_ ┆ category_ ┆ category_ ┆ … ┆ unit_cost ┆ category_ ┆ category_ ┆ category │
│ d         ┆ major_cd  ┆ medium_cd ┆ small_cd  ┆   ┆ ---       ┆ major_nam ┆ medium_na ┆ _small_n │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ i64       ┆ e         ┆ me        ┆ ame      │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆           ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆ str       ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ P04010100 ┆ 04        ┆ 0401      ┆ 040101    ┆ … ┆ 149       ┆ 惣菜      ┆ 御飯類    ┆ 弁当類   │
│ 1         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ P04010100 ┆ 04        ┆ 0401      ┆ 040101    ┆ … ┆ 164       ┆ 惣菜      ┆ 御飯類 

---
> P-094: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [169]:
let path_094 = "../data/P_df_product_full_UTF-8_header.csv";
let mut f = std::fs::File::create(&path_094)?;
CsvWriter::new(&mut f)
    .has_header(true)
    .with_delimiter(b",")
    .finish(&mut df_product_full.clone().collect().unwrap())?;

---
> P-095: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|CP932|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [160]:
let path = "../data/P_df_product_full_cp932_header.csv";
let mut f = std::fs::File::create(path)?;
CsvWriter::new(&mut f)
    .has_header(true)
    .with_delimiter(b",")
    .finish(&mut df_product_full.clone().collect().unwrap())?;

---
> P-096: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|無し|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること。
> 
> |出力先|
> |:--:|
> |./data|

In [170]:
let path_096 = "../data/P_df_product_full_UTF-8_noh.csv";
let mut f = std::fs::File::create(&path_096)?;
CsvWriter::new(&mut f)
    .has_header(false)
    .with_delimiter(b",")
    .finish(&mut df_product_full.clone().collect().unwrap())?;

---
> P-097: 094で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|有り|UTF-8|

In [168]:
let path_094 = "../data/P_df_product_full_UTF-8_header.csv";
let df_product_full2 = LazyCsvReader::new(&path_094)
    .has_header(true)
    .with_dtype_overwrite(Some(&schema.clone()))
    .finish()
    .unwrap();

df_product_full2.collect().unwrap().head(Some(3))

shape: (3, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ product_c ┆ category_ ┆ category_ ┆ category_ ┆ … ┆ unit_cost ┆ category_ ┆ category_ ┆ category │
│ d         ┆ major_cd  ┆ medium_cd ┆ small_cd  ┆   ┆ ---       ┆ major_nam ┆ medium_na ┆ _small_n │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ i64       ┆ e         ┆ me        ┆ ame      │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆           ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆ str       ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ P04010100 ┆ 04        ┆ 0401      ┆ 040101    ┆ … ┆ 149       ┆ 惣菜      ┆ 御飯類    ┆ 弁当類   │
│ 1         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ P04010100 ┆ 04        ┆ 0401      ┆ 040101    ┆ … ┆ 164       ┆ 惣菜      ┆ 御飯類    ┆ 

---
> P-098: 096で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |CSV（カンマ区切り）|ヘッダ無し|UTF-8|

In [199]:
// データ型の指定
let mut new_schema = Schema::new();
new_schema.with_column("column_1".into(), DataType::Utf8);
new_schema.with_column("column_2".into(), DataType::Utf8);
new_schema.with_column("column_3".into(), DataType::Utf8);
new_schema.with_column("column_4".into(), DataType::Utf8);

let old_c_names = vec![
    "column_1", 
    "column_2",
    "column_3",
    "column_4",
    "column_5",
    "column_6",
    "column_7",
    "column_8",
    "column_9",
];

let new_c_names = vec![
    "product_cd", 
    "category_major_cd",
    "category_medium_cd",
    "category_small_cd",
    "unit_price",
    "unit_cost",
    "category_major_name",
    "category_medium_name",
    "category_small_name",
];

let path_096 = "../data/P_df_product_full_UTF-8_noh.csv";
let df_product_full3: LazyFrame = LazyCsvReader::new(&path_096)
    .has_header(false)
    .with_dtype_overwrite(Some(&new_schema.clone()))
    .finish()?
    .rename(old_c_names, new_c_names);

df_product_full3.collect().unwrap().head(Some(3))

shape: (3, 9)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ product_c ┆ category_ ┆ category_ ┆ category_ ┆ … ┆ unit_cost ┆ category_ ┆ category_ ┆ category │
│ d         ┆ major_cd  ┆ medium_cd ┆ small_cd  ┆   ┆ ---       ┆ major_nam ┆ medium_na ┆ _small_n │
│ ---       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ i64       ┆ e         ┆ me        ┆ ame      │
│ str       ┆ str       ┆ str       ┆ str       ┆   ┆           ┆ ---       ┆ ---       ┆ ---      │
│           ┆           ┆           ┆           ┆   ┆           ┆ str       ┆ str       ┆ str      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ P04010100 ┆ 04        ┆ 0401      ┆ 040101    ┆ … ┆ 149       ┆ 惣菜      ┆ 御飯類    ┆ 弁当類   │
│ 1         ┆           ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│ P04010100 ┆ 04        ┆ 0401      ┆ 040101    ┆ … ┆ 164       ┆ 惣菜      ┆ 御飯類    ┆ 

---
> P-099: 093で作成したカテゴリ名付き商品データを以下の仕様でファイル出力せよ。
>
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|
> 
> ファイル出力先のパスは以下のようにすること
> 
> |出力先|
> |:--:|
> |./data|

In [219]:
let path_099 = "../data/P_df_product_full_UTF-8_header.tsv";
let mut f = std::fs::File::create(&path_099)?;
CsvWriter::new(&mut f)
    .has_header(true)
    .with_delimiter(b"\x09")
    .finish(&mut df_product_full.clone().collect().unwrap())?;

Error: invalid character in numeric character escape: `\`

---
> P-100: 099で作成した以下形式のファイルを読み込み、データを3件を表示させて正しく取り込まれていることを確認せよ。
> 
> |ファイル形式|ヘッダ有無|文字エンコーディング|
> |:--:|:--:|:--:|
> |TSV（タブ区切り）|有り|UTF-8|

In [ ]:
let path_099 = "../data/P_df_product_full_UTF-8_header.tsv";
let df_product_full4 = LazyCsvReader::new(&path_099)
    .has_header(true)
    .with_delimiter(b"\x09")
    .with_dtype_overwrite(Some(&schema.clone()))
    .finish()
    .unwrap();

df_product_full4.collect().unwrap().head(Some(3))

# これで１００本終わりです。おつかれさまでした！